<a href="https://colab.research.google.com/github/JarredParrettDickinson/AG_DATA_ANALYSIS/blob/master/STATE_ECONOMIC_DATA_SUMMARY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Welcome to my widget to present State Economic Agricultre Data Geographically.
## The following steps will direct you to operate the notebook if you are not familiar. 


1.   On the header of the Google CoLab, select `Runtime` -> `Run All`
2.   Scroll down to `Using the interactive portion` below to plot on a map 






In [0]:
import pandas as pd
import plotly
import plotly.express as px
from gzip import GzipFile
import requests
from urllib.request import Request, urlopen
import urllib.request as ur
import numpy as np
import requests
import json
from pandas.io.json import json_normalize 
import io
from ipywidgets import widgets
import math
from google.colab import drive
import urllib
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
from IPython.display import display
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
import datetime as dt
from urllib.request import urlopen

In [0]:
def get_ftp_file_names_economic():
    #get file names
    try:
        import urllib.request as urllib2
    except ImportError:
        import urllib2
    import re
    urlpath =urlopen('ftp://ftp.nass.usda.gov/quickstats')
    string = urlpath.read().decode('utf-8')
    split = string.split('\n')
    file_names = []
    for line in split:
      sub_split = line.split(' ')
      name = sub_split[-1]
      file_names.append(name)
    for name in file_names:
      if "economic" in name:
          print(name)
          return name

def pull_files(file_name):
  file_name = file_name[0:-1]
  ftp_path ='ftp://ftp.nass.usda.gov/quickstats/'+str(file_name)
  return ftp_path

In [0]:
#Get Data from USDA FTP server - this may take a minute
file_name = get_ftp_file_names_economic()
url = pull_files(file_name)
print(url)
req = ur.Request(url) #  gz file on ftp (ensure startswith 'ftp://')
z_f = ur.urlopen(req)
f = GzipFile(fileobj=z_f, mode="r")
df = pd.read_csv(f, sep='\t')

In [0]:
#store df to not have to pull again
store_df = df
df=store_df
len(df)

# Select State by changing it too the value below to the states initials
## Note at this has been updated and may now be done with any state



In [0]:
STATE = "CA"

In [0]:
  df_economic = df.loc[((df["STATE_ALPHA"] == STATE))]

In [0]:
dict_states={'AL': {'AUTAUGA': '01001', 'BALDWIN': '01003', 'BARBOUR': '01005', 'BIBB': '01007', 'BLOUNT': '01009', 'BULLOCK': '01011', 'BUTLER': '01013', 'CALHOUN': '01015', 'CHAMBERS': '01017', 'CHEROKEE': '01019', 'CHILTON': '01021', 'CHOCTAW': '01023', 'CLARKE': '01025', 'CLAY': '01027', 'CLEBURNE': '01029', 'COFFEE': '01031', 'COLBERT': '01033', 'CONECUH': '01035', 'COOSA': '01037', 'COVINGTON': '01039', 'CRENSHAW': '01041', 'CULLMAN': '01043', 'DALE': '01045', 'DALLAS': '01047', 'DE KALB': '01049', 'ELMORE': '01051', 'ESCAMBIA': '01053', 'ETOWAH': '01055', 'FAYETTE': '01057', 'FRANKLIN': '01059', 'GENEVA': '01061', 'GREENE': '01063', 'HALE': '01065', 'HENRY': '01067', 'HOUSTON': '01069', 'JACKSON': '01071', 'JEFFERSON': '01073', 'LAMAR': '01075', 'LAUDERDALE': '01077', 'LAWRENCE': '01079', 'LEE': '01081', 'LIMESTONE': '01083', 'LOWNDES': '01085', 'MACON': '01087', 'MADISON': '01089', 'MARENGO': '01091', 'MARION': '01093', 'MARSHALL': '01095', 'MOBILE': '01097', 'MONROE': '01099', 'MONTGOMERY': '01101', 'MORGAN': '01103', 'PERRY': '01105', 'PICKENS': '01107', 'PIKE': '01109', 'RANDOLPH': '01111', 'RUSSELL': '01113', 'ST CLAIR': '01115', 'SHELBY': '01117', 'SUMTER': '01119', 'TALLADEGA': '01121', 'TALLAPOOSA': '01123', 'TUSCALOOSA': '01125', 'WALKER': '01127', 'WASHINGTON': '01129', 'WILCOX': '01131', 'WINSTON': '01133'}, 'AK': {'ALEUTIANS EAST': '02013', 'ALEUTIANS WEST': '02016', 'ANCHORAGE': '02020', 'BETHEL': '02050', 'BRISTOL BAY': '02060', 'DENALI': '02068', 'DILLINGHAM': '02070', 'FAIRBANKS NORTH STAR': '02090', 'HAINES': '02100', 'JUNEAU': '02110', 'KENAI PENINSULA': '02122', 'KETCHIKAN GATEWAY': '02130', 'KODIAK ISLAND': '02150', 'LAKE AND PENINSULA': '02164', 'MATANUSKA-SUSITNA': '02170', 'NOME': '02180', 'NORTH SLOPE': '02185', 'NORTHWEST ARCTIC': '02188', 'PRINCE OF WALES-OUTER KETCHIKAN': '02201', 'SITKA': '02220', 'SKAGWAY-HOONAH-ANGOON': '02232', 'SOUTHEAST FAIRBANKS': '02240', 'VALDEZ-CORDOVA': '02261', 'WADE HAMPTON': '02270', 'WRANGELL-PETERSBURG': '02280', 'YAKUTAT': '02282', 'YUKON-KOYUKUK': '02290'}, 'AZ': {'APACHE': '04001', 'COCHISE': '04003', 'COCONINO': '04005', 'GILA': '04007', 'GRAHAM': '04009', 'GREENLEE': '04011', 'LA PAZ': '04012', 'MARICOPA': '04013', 'MOHAVE': '04015', 'NAVAJO': '04017', 'PIMA': '04019', 'PINAL': '04021', 'SANTA CRUZ': '04023', 'YAVAPAI': '04025', 'YUMA': '04027'}, 'AR': {'ARKANSAS': '05001', 'ASHLEY': '05003', 'BAXTER': '05005', 'BENTON': '05007', 'BOONE': '05009', 'BRADLEY': '05011', 'CALHOUN': '05013', 'CARROLL': '05015', 'CHICOT': '05017', 'CLARK': '05019', 'CLAY': '05021', 'CLEBURNE': '05023', 'CLEVELAND': '05025', 'COLUMBIA': '05027', 'CONWAY': '05029', 'CRAIGHEAD': '05031', 'CRAWFORD': '05033', 'CRITTENDEN': '05035', 'CROSS': '05037', 'DALLAS': '05039', 'DESHA': '05041', 'DREW': '05043', 'FAULKNER': '05045', 'FRANKLIN': '05047', 'FULTON': '05049', 'GARLAND': '05051', 'GRANT': '05053', 'GREENE': '05055', 'HEMPSTEAD': '05057', 'HOT SPRING': '05059', 'HOWARD': '05061', 'INDEPENDENCE': '05063', 'IZARD': '05065', 'JACKSON': '05067', 'JEFFERSON': '05069', 'JOHNSON': '05071', 'LAFAYETTE': '05073', 'LAWRENCE': '05075', 'LEE': '05077', 'LINCOLN': '05079', 'LITTLE RIVER': '05081', 'LOGAN': '05083', 'LONOKE': '05085', 'MADISON': '05087', 'MARION': '05089', 'MILLER': '05091', 'MISSISSIPPI': '05093', 'MONROE': '05095', 'MONTGOMERY': '05097', 'NEVADA': '05099', 'NEWTON': '05101', 'OUACHITA': '05103', 'PERRY': '05105', 'PHILLIPS': '05107', 'PIKE': '05109', 'POINSETT': '05111', 'POLK': '05113', 'POPE': '05115', 'PRAIRIE': '05117', 'PULASKI': '05119', 'RANDOLPH': '05121', 'ST FRANCIS': '05123', 'SALINE': '05125', 'SCOTT': '05127', 'SEARCY': '05129', 'SEBASTIAN': '05131', 'SEVIER': '05133', 'SHARP': '05135', 'STONE': '05137', 'UNION': '05139', 'VAN BUREN': '05141', 'WASHINGTON': '05143', 'WHITE': '05145', 'WOODRUFF': '05147', 'YELL': '05149'}, 'CA': {'ALAMEDA': '06001', 'ALPINE': '06003', 'AMADOR': '06005', 'BUTTE': '06007', 'CALAVERAS': '06009', 'COLUSA': '06011', 'CONTRA COSTA': '06013', 'DEL NORTE': '06015', 'EL DORADO': '06017', 'FRESNO': '06019', 'GLENN': '06021', 'HUMBOLDT': '06023', 'IMPERIAL': '06025', 'INYO': '06027', 'KERN': '06029', 'KINGS': '06031', 'LAKE': '06033', 'LASSEN': '06035', 'LOS ANGELES': '06037', 'MADERA': '06039', 'MARIN': '06041', 'MARIPOSA': '06043', 'MENDOCINO': '06045', 'MERCED': '06047', 'MODOC': '06049', 'MONO': '06051', 'MONTEREY': '06053', 'NAPA': '06055', 'NEVADA': '06057', 'ORANGE': '06059', 'PLACER': '06061', 'PLUMAS': '06063', 'RIVERSIDE': '06065', 'SACRAMENTO': '06067', 'SAN BENITO': '06069', 'SAN BERNARDINO': '06071', 'SAN DIEGO': '06073', 'SAN FRANCISCO': '06075', 'SAN JOAQUIN': '06077', 'SAN LUIS OBISPO': '06079', 'SAN MATEO': '06081', 'SANTA BARBARA': '06083', 'SANTA CLARA': '06085', 'SANTA CRUZ': '06087', 'SHASTA': '06089', 'SIERRA': '06091', 'SISKIYOU': '06093', 'SOLANO': '06095', 'SONOMA': '06097', 'STANISLAUS': '06099', 'SUTTER': '06101', 'TEHAMA': '06103', 'TRINITY': '06105', 'TULARE': '06107', 'TUOLUMNE': '06109', 'VENTURA': '06111', 'YOLO': '06113', 'YUBA': '06115'}, 'CO': {'ADAMS': '08001', 'ALAMOSA': '08003', 'ARAPAHOE': '08005', 'ARCHULETA': '08007', 'BACA': '08009', 'BENT': '08011', 'BOULDER': '08013', 'CHAFFEE': '08015', 'CHEYENNE': '08017', 'CLEAR CREEK': '08019', 'CONEJOS': '08021', 'COSTILLA': '08023', 'CROWLEY': '08025', 'CUSTER': '08027', 'DELTA': '08029', 'DENVER': '08031', 'DOLORES': '08033', 'DOUGLAS': '08035', 'EAGLE': '08037', 'ELBERT': '08039', 'EL PASO': '08041', 'FREMONT': '08043', 'GARFIELD': '08045', 'GILPIN': '08047', 'GRAND': '08049', 'GUNNISON': '08051', 'HINSDALE': '08053', 'HUERFANO': '08055', 'JACKSON': '08057', 'JEFFERSON': '08059', 'KIOWA': '08061', 'KIT CARSON': '08063', 'LAKE': '08065', 'LA PLATA': '08067', 'LARIMER': '08069', 'LAS ANIMAS': '08071', 'LINCOLN': '08073', 'LOGAN': '08075', 'MESA': '08077', 'MINERAL': '08079', 'MOFFAT': '08081', 'MONTEZUMA': '08083', 'MONTROSE': '08085', 'MORGAN': '08087', 'OTERO': '08089', 'OURAY': '08091', 'PARK': '08093', 'PHILLIPS': '08095', 'PITKIN': '08097', 'PROWERS': '08099', 'PUEBLO': '08101', 'RIO BLANCO': '08103', 'RIO GRANDE': '08105', 'ROUTT': '08107', 'SAGUACHE': '08109', 'SAN JUAN': '08111', 'SAN MIGUEL': '08113', 'SEDGWICK': '08115', 'SUMMIT': '08117', 'TELLER': '08119', 'WASHINGTON': '08121', 'WELD': '08123', 'YUMA': '08125'}, 'CT': {'FAIRFIELD': '09001', 'HARTFORD': '09003', 'LITCHFIELD': '09005', 'MIDDLESEX': '09007', 'NEW HAVEN': '09009', 'NEW LONDON': '09011', 'TOLLAND': '09013', 'WINDHAM': '09015'}, 'DE': {'KENT': '10001', 'NEW CASTLE': '10003', 'SUSSEX': '10005'}, 'DC': {'WASHINGTON': '11001'}, 'FL': {'ALACHUA': '12001', 'BAKER': '12003', 'BAY': '12005', 'BRADFORD': '12007', 'BREVARD': '12009', 'BROWARD': '12011', 'CALHOUN': '12013', 'CHARLOTTE': '12015', 'CITRUS': '12017', 'CLAY': '12019', 'COLLIER': '12021', 'COLUMBIA': '12023', 'DE SOTO': '12027', 'DIXIE': '12029', 'DUVAL': '12031', 'ESCAMBIA': '12033', 'FLAGLER': '12035', 'FRANKLIN': '12037', 'GADSDEN': '12039', 'GILCHRIST': '12041', 'GLADES': '12043', 'GULF': '12045', 'HAMILTON': '12047', 'HARDEE': '12049', 'HENDRY': '12051', 'HERNANDO': '12053', 'HIGHLANDS': '12055', 'HILLSBOROUGH': '12057', 'HOLMES': '12059', 'INDIAN RIVER': '12061', 'JACKSON': '12063', 'JEFFERSON': '12065', 'LAFAYETTE': '12067', 'LAKE': '12069', 'LEE': '12071', 'LEON': '12073', 'LEVY': '12075', 'LIBERTY': '12077', 'MADISON': '12079', 'MANATEE': '12081', 'MARION': '12083', 'MARTIN': '12085', 'MIAMI-DADE': '12086', 'MONROE': '12087', 'NASSAU': '12089', 'OKALOOSA': '12091', 'OKEECHOBEE': '12093', 'ORANGE': '12095', 'OSCEOLA': '12097', 'PALM BEACH': '12099', 'PASCO': '12101', 'PINELLAS': '12103', 'POLK': '12105', 'PUTNAM': '12107', 'ST JOHNS': '12109', 'ST LUCIE': '12111', 'SANTA ROSA': '12113', 'SARASOTA': '12115', 'SEMINOLE': '12117', 'SUMTER': '12119', 'SUWANNEE': '12121', 'TAYLOR': '12123', 'UNION': '12125', 'VOLUSIA': '12127', 'WAKULLA': '12129', 'WALTON': '12131', 'WASHINGTON': '12133'}, 'GA': {'APPLING': '13001', 'ATKINSON': '13003', 'BACON': '13005', 'BAKER': '13007', 'BALDWIN': '13009', 'BANKS': '13011', 'BARROW': '13013', 'BARTOW': '13015', 'BEN HILL': '13017', 'BERRIEN': '13019', 'BIBB': '13021', 'BLECKLEY': '13023', 'BRANTLEY': '13025', 'BROOKS': '13027', 'BRYAN': '13029', 'BULLOCH': '13031', 'BURKE': '13033', 'BUTTS': '13035', 'CALHOUN': '13037', 'CAMDEN': '13039', 'CANDLER': '13043', 'CARROLL': '13045', 'CATOOSA': '13047', 'CHARLTON': '13049', 'CHATHAM': '13051', 'CHATTAHOOCHEE': '13053', 'CHATTOOGA': '13055', 'CHEROKEE': '13057', 'CLARKE': '13059', 'CLAY': '13061', 'CLAYTON': '13063', 'CLINCH': '13065', 'COBB': '13067', 'COFFEE': '13069', 'COLQUITT': '13071', 'COLUMBIA': '13073', 'COOK': '13075', 'COWETA': '13077', 'CRAWFORD': '13079', 'CRISP': '13081', 'DADE': '13083', 'DAWSON': '13085', 'DECATUR': '13087', 'DE KALB': '13089', 'DODGE': '13091', 'DOOLY': '13093', 'DOUGHERTY': '13095', 'DOUGLAS': '13097', 'EARLY': '13099', 'ECHOLS': '13101', 'EFFINGHAM': '13103', 'ELBERT': '13105', 'EMANUEL': '13107', 'EVANS': '13109', 'FANNIN': '13111', 'FAYETTE': '13113', 'FLOYD': '13115', 'FORSYTH': '13117', 'FRANKLIN': '13119', 'FULTON': '13121', 'GILMER': '13123', 'GLASCOCK': '13125', 'GLYNN': '13127', 'GORDON': '13129', 'GRADY': '13131', 'GREENE': '13133', 'GWINNETT': '13135', 'HABERSHAM': '13137', 'HALL': '13139', 'HANCOCK': '13141', 'HARALSON': '13143', 'HARRIS': '13145', 'HART': '13147', 'HEARD': '13149', 'HENRY': '13151', 'HOUSTON': '13153', 'IRWIN': '13155', 'JACKSON': '13157', 'JASPER': '13159', 'JEFF DAVIS': '13161', 'JEFFERSON': '13163', 'JENKINS': '13165', 'JOHNSON': '13167', 'JONES': '13169', 'LAMAR': '13171', 'LANIER': '13173', 'LAURENS': '13175', 'LEE': '13177', 'LIBERTY': '13179', 'LINCOLN': '13181', 'LONG': '13183', 'LOWNDES': '13185', 'LUMPKIN': '13187', 'MCDUFFIE': '13189', 'MCINTOSH': '13191', 'MACON': '13193', 'MADISON': '13195', 'MARION': '13197', 'MERIWETHER': '13199', 'MILLER': '13201', 'MITCHELL': '13205', 'MONROE': '13207', 'MONTGOMERY': '13209', 'MORGAN': '13211', 'MURRAY': '13213', 'MUSCOGEE': '13215', 'NEWTON': '13217', 'OCONEE': '13219', 'OGLETHORPE': '13221', 'PAULDING': '13223', 'PEACH': '13225', 'PICKENS': '13227', 'PIERCE': '13229', 'PIKE': '13231', 'POLK': '13233', 'PULASKI': '13235', 'PUTNAM': '13237', 'QUITMAN': '13239', 'RABUN': '13241', 'RANDOLPH': '13243', 'RICHMOND': '13245', 'ROCKDALE': '13247', 'SCHLEY': '13249', 'SCREVEN': '13251', 'SEMINOLE': '13253', 'SPALDING': '13255', 'STEPHENS': '13257', 'STEWART': '13259', 'SUMTER': '13261', 'TALBOT': '13263', 'TALIAFERRO': '13265', 'TATTNALL': '13267', 'TAYLOR': '13269', 'TELFAIR': '13271', 'TERRELL': '13273', 'THOMAS': '13275', 'TIFT': '13277', 'TOOMBS': '13279', 'TOWNS': '13281', 'TREUTLEN': '13283', 'TROUP': '13285', 'TURNER': '13287', 'TWIGGS': '13289', 'UNION': '13291', 'UPSON': '13293', 'WALKER': '13295', 'WALTON': '13297', 'WARE': '13299', 'WARREN': '13301', 'WASHINGTON': '13303', 'WAYNE': '13305', 'WEBSTER': '13307', 'WHEELER': '13309', 'WHITE': '13311', 'WHITFIELD': '13313', 'WILCOX': '13315', 'WILKES': '13317', 'WILKINSON': '13319', 'WORTH': '13321'}, 'HI': {'HAWAII': '15001', 'HONOLULU': '15003', 'KAUAI': '15007', 'MAUI': '15009'}, 'ID': {'ADA': '16001', 'ADAMS': '16003', 'BANNOCK': '16005', 'BEAR LAKE': '16007', 'BENEWAH': '16009', 'BINGHAM': '16011', 'BLAINE': '16013', 'BOISE': '16015', 'BONNER': '16017', 'BONNEVILLE': '16019', 'BOUNDARY': '16021', 'BUTTE': '16023', 'CAMAS': '16025', 'CANYON': '16027', 'CARIBOU': '16029', 'CASSIA': '16031', 'CLARK': '16033', 'CLEARWATER': '16035', 'CUSTER': '16037', 'ELMORE': '16039', 'FRANKLIN': '16041', 'FREMONT': '16043', 'GEM': '16045', 'GOODING': '16047', 'IDAHO': '16049', 'JEFFERSON': '16051', 'JEROME': '16053', 'KOOTENAI': '16055', 'LATAH': '16057', 'LEMHI': '16059', 'LEWIS': '16061', 'LINCOLN': '16063', 'MADISON': '16065', 'MINIDOKA': '16067', 'NEZ PERCE': '16069', 'ONEIDA': '16071', 'OWYHEE': '16073', 'PAYETTE': '16075', 'POWER': '16077', 'SHOSHONE': '16079', 'TETON': '16081', 'TWIN FALLS': '16083', 'VALLEY': '16085', 'WASHINGTON': '16087'}, 'IL': {'ADAMS': '17001', 'ALEXANDER': '17003', 'BOND': '17005', 'BOONE': '17007', 'BROWN': '17009', 'BUREAU': '17011', 'CALHOUN': '17013', 'CARROLL': '17015', 'CASS': '17017', 'CHAMPAIGN': '17019', 'CHRISTIAN': '17021', 'CLARK': '17023', 'CLAY': '17025', 'CLINTON': '17027', 'COLES': '17029', 'COOK': '17031', 'CRAWFORD': '17033', 'CUMBERLAND': '17035', 'DE KALB': '17037', 'DE WITT': '17039', 'DOUGLAS': '17041', 'DU PAGE': '17043', 'EDGAR': '17045', 'EDWARDS': '17047', 'EFFINGHAM': '17049', 'FAYETTE': '17051', 'FORD': '17053', 'FRANKLIN': '17055', 'FULTON': '17057', 'GALLATIN': '17059', 'GREENE': '17061', 'GRUNDY': '17063', 'HAMILTON': '17065', 'HANCOCK': '17067', 'HARDIN': '17069', 'HENDERSON': '17071', 'HENRY': '17073', 'IROQUOIS': '17075', 'JACKSON': '17077', 'JASPER': '17079', 'JEFFERSON': '17081', 'JERSEY': '17083', 'JO DAVIESS': '17085', 'JOHNSON': '17087', 'KANE': '17089', 'KANKAKEE': '17091', 'KENDALL': '17093', 'KNOX': '17095', 'LAKE': '17097', 'LA SALLE': '17099', 'LAWRENCE': '17101', 'LEE': '17103', 'LIVINGSTON': '17105', 'LOGAN': '17107', 'MCDONOUGH': '17109', 'MCHENRY': '17111', 'MCLEAN': '17113', 'MACON': '17115', 'MACOUPIN': '17117', 'MADISON': '17119', 'MARION': '17121', 'MARSHALL': '17123', 'MASON': '17125', 'MASSAC': '17127', 'MENARD': '17129', 'MERCER': '17131', 'MONROE': '17133', 'MONTGOMERY': '17135', 'MORGAN': '17137', 'MOULTRIE': '17139', 'OGLE': '17141', 'PEORIA': '17143', 'PERRY': '17145', 'PIATT': '17147', 'PIKE': '17149', 'POPE': '17151', 'PULASKI': '17153', 'PUTNAM': '17155', 'RANDOLPH': '17157', 'RICHLAND': '17159', 'ROCK ISLAND': '17161', 'ST CLAIR': '17163', 'SALINE': '17165', 'SANGAMON': '17167', 'SCHUYLER': '17169', 'SCOTT': '17171', 'SHELBY': '17173', 'STARK': '17175', 'STEPHENSON': '17177', 'TAZEWELL': '17179', 'UNION': '17181', 'VERMILION': '17183', 'WABASH': '17185', 'WARREN': '17187', 'WASHINGTON': '17189', 'WAYNE': '17191', 'WHITE': '17193', 'WHITESIDE': '17195', 'WILL': '17197', 'WILLIAMSON': '17199', 'WINNEBAGO': '17201', 'WOODFORD': '17203'}, 'IN': {'ADAMS': '18001', 'ALLEN': '18003', 'BARTHOLOMEW': '18005', 'BENTON': '18007', 'BLACKFORD': '18009', 'BOONE': '18011', 'BROWN': '18013', 'CARROLL': '18015', 'CASS': '18017', 'CLARK': '18019', 'CLAY': '18021', 'CLINTON': '18023', 'CRAWFORD': '18025', 'DAVIESS': '18027', 'DEARBORN': '18029', 'DECATUR': '18031', 'DE KALB': '18033', 'DELAWARE': '18035', 'DUBOIS': '18037', 'ELKHART': '18039', 'FAYETTE': '18041', 'FLOYD': '18043', 'FOUNTAIN': '18045', 'FRANKLIN': '18047', 'FULTON': '18049', 'GIBSON': '18051', 'GRANT': '18053', 'GREENE': '18055', 'HAMILTON': '18057', 'HANCOCK': '18059', 'HARRISON': '18061', 'HENDRICKS': '18063', 'HENRY': '18065', 'HOWARD': '18067', 'HUNTINGTON': '18069', 'JACKSON': '18071', 'JASPER': '18073', 'JAY': '18075', 'JEFFERSON': '18077', 'JENNINGS': '18079', 'JOHNSON': '18081', 'KNOX': '18083', 'KOSCIUSKO': '18085', 'LA GRANGE': '18087', 'LAKE': '18089', 'LA PORTE': '18091', 'LAWRENCE': '18093', 'MADISON': '18095', 'MARION': '18097', 'MARSHALL': '18099', 'MARTIN': '18101', 'MIAMI': '18103', 'MONROE': '18105', 'MONTGOMERY': '18107', 'MORGAN': '18109', 'NEWTON': '18111', 'NOBLE': '18113', 'OHIO': '18115', 'ORANGE': '18117', 'OWEN': '18119', 'PARKE': '18121', 'PERRY': '18123', 'PIKE': '18125', 'PORTER': '18127', 'POSEY': '18129', 'PULASKI': '18131', 'PUTNAM': '18133', 'RANDOLPH': '18135', 'RIPLEY': '18137', 'RUSH': '18139', 'ST JOSEPH': '18141', 'SCOTT': '18143', 'SHELBY': '18145', 'SPENCER': '18147', 'STARKE': '18149', 'STEUBEN': '18151', 'SULLIVAN': '18153', 'SWITZERLAND': '18155', 'TIPPECANOE': '18157', 'TIPTON': '18159', 'UNION': '18161', 'VANDERBURGH': '18163', 'VERMILLION': '18165', 'VIGO': '18167', 'WABASH': '18169', 'WARREN': '18171', 'WARRICK': '18173', 'WASHINGTON': '18175', 'WAYNE': '18177', 'WELLS': '18179', 'WHITE': '18181', 'WHITLEY': '18183'}, 'IA': {'ADAIR': '19001', 'ADAMS': '19003', 'ALLAMAKEE': '19005', 'APPANOOSE': '19007', 'AUDUBON': '19009', 'BENTON': '19011', 'BLACK HAWK': '19013', 'BOONE': '19015', 'BREMER': '19017', 'BUCHANAN': '19019', 'BUENA VISTA': '19021', 'BUTLER': '19023', 'CALHOUN': '19025', 'CARROLL': '19027', 'CASS': '19029', 'CEDAR': '19031', 'CERRO GORDO': '19033', 'CHEROKEE': '19035', 'CHICKASAW': '19037', 'CLARKE': '19039', 'CLAY': '19041', 'CLAYTON': '19043', 'CLINTON': '19045', 'CRAWFORD': '19047', 'DALLAS': '19049', 'DAVIS': '19051', 'DECATUR': '19053', 'DELAWARE': '19055', 'DES MOINES': '19057', 'DICKINSON': '19059', 'DUBUQUE': '19061', 'EMMET': '19063', 'FAYETTE': '19065', 'FLOYD': '19067', 'FRANKLIN': '19069', 'FREMONT': '19071', 'GREENE': '19073', 'GRUNDY': '19075', 'GUTHRIE': '19077', 'HAMILTON': '19079', 'HANCOCK': '19081', 'HARDIN': '19083', 'HARRISON': '19085', 'HENRY': '19087', 'HOWARD': '19089', 'HUMBOLDT': '19091', 'IDA': '19093', 'IOWA': '19095', 'JACKSON': '19097', 'JASPER': '19099', 'JEFFERSON': '19101', 'JOHNSON': '19103', 'JONES': '19105', 'KEOKUK': '19107', 'KOSSUTH': '19109', 'LEE': '19111', 'LINN': '19113', 'LOUISA': '19115', 'LUCAS': '19117', 'LYON': '19119', 'MADISON': '19121', 'MAHASKA': '19123', 'MARION': '19125', 'MARSHALL': '19127', 'MILLS': '19129', 'MITCHELL': '19131', 'MONONA': '19133', 'MONROE': '19135', 'MONTGOMERY': '19137', 'MUSCATINE': '19139', 'O BRIEN': '19141', 'OSCEOLA': '19143', 'PAGE': '19145', 'PALO ALTO': '19147', 'PLYMOUTH': '19149', 'POCAHONTAS': '19151', 'POLK': '19153', 'POTTAWATTAMIE': '19155', 'POWESHIEK': '19157', 'RINGGOLD': '19159', 'SAC': '19161', 'SCOTT': '19163', 'SHELBY': '19165', 'SIOUX': '19167', 'STORY': '19169', 'TAMA': '19171', 'TAYLOR': '19173', 'UNION': '19175', 'VAN BUREN': '19177', 'WAPELLO': '19179', 'WARREN': '19181', 'WASHINGTON': '19183', 'WAYNE': '19185', 'WEBSTER': '19187', 'WINNEBAGO': '19189', 'WINNESHIEK': '19191', 'WOODBURY': '19193', 'WORTH': '19195', 'WRIGHT': '19197'}, 'KS': {'ALLEN': '20001', 'ANDERSON': '20003', 'ATCHISON': '20005', 'BARBER': '20007', 'BARTON': '20009', 'BOURBON': '20011', 'BROWN': '20013', 'BUTLER': '20015', 'CHASE': '20017', 'CHAUTAUQUA': '20019', 'CHEROKEE': '20021', 'CHEYENNE': '20023', 'CLARK': '20025', 'CLAY': '20027', 'CLOUD': '20029', 'COFFEY': '20031', 'COMANCHE': '20033', 'COWLEY': '20035', 'CRAWFORD': '20037', 'DECATUR': '20039', 'DICKINSON': '20041', 'DONIPHAN': '20043', 'DOUGLAS': '20045', 'EDWARDS': '20047', 'ELK': '20049', 'ELLIS': '20051', 'ELLSWORTH': '20053', 'FINNEY': '20055', 'FORD': '20057', 'FRANKLIN': '20059', 'GEARY': '20061', 'GOVE': '20063', 'GRAHAM': '20065', 'GRANT': '20067', 'GRAY': '20069', 'GREELEY': '20071', 'GREENWOOD': '20073', 'HAMILTON': '20075', 'HARPER': '20077', 'HARVEY': '20079', 'HASKELL': '20081', 'HODGEMAN': '20083', 'JACKSON': '20085', 'JEFFERSON': '20087', 'JEWELL': '20089', 'JOHNSON': '20091', 'KEARNY': '20093', 'KINGMAN': '20095', 'KIOWA': '20097', 'LABETTE': '20099', 'LANE': '20101', 'LEAVENWORTH': '20103', 'LINCOLN': '20105', 'LINN': '20107', 'LOGAN': '20109', 'LYON': '20111', 'MCPHERSON': '20113', 'MARION': '20115', 'MARSHALL': '20117', 'MEADE': '20119', 'MIAMI': '20121', 'MITCHELL': '20123', 'MONTGOMERY': '20125', 'MORRIS': '20127', 'MORTON': '20129', 'NEMAHA': '20131', 'NEOSHO': '20133', 'NESS': '20135', 'NORTON': '20137', 'OSAGE': '20139', 'OSBORNE': '20141', 'OTTAWA': '20143', 'PAWNEE': '20145', 'PHILLIPS': '20147', 'POTTAWATOMIE': '20149', 'PRATT': '20151', 'RAWLINS': '20153', 'RENO': '20155', 'REPUBLIC': '20157', 'RICE': '20159', 'RILEY': '20161', 'ROOKS': '20163', 'RUSH': '20165', 'RUSSELL': '20167', 'SALINE': '20169', 'SCOTT': '20171', 'SEDGWICK': '20173', 'SEWARD': '20175', 'SHAWNEE': '20177', 'SHERIDAN': '20179', 'SHERMAN': '20181', 'SMITH': '20183', 'STAFFORD': '20185', 'STANTON': '20187', 'STEVENS': '20189', 'SUMNER': '20191', 'THOMAS': '20193', 'TREGO': '20195', 'WABAUNSEE': '20197', 'WALLACE': '20199', 'WASHINGTON': '20201', 'WICHITA': '20203', 'WILSON': '20205', 'WOODSON': '20207', 'WYANDOTTE': '20209'}, 'KY': {'ADAIR': '21001', 'ALLEN': '21003', 'ANDERSON': '21005', 'BALLARD': '21007', 'BARREN': '21009', 'BATH': '21011', 'BELL': '21013', 'BOONE': '21015', 'BOURBON': '21017', 'BOYD': '21019', 'BOYLE': '21021', 'BRACKEN': '21023', 'BREATHITT': '21025', 'BRECKINRIDGE': '21027', 'BULLITT': '21029', 'BUTLER': '21031', 'CALDWELL': '21033', 'CALLOWAY': '21035', 'CAMPBELL': '21037', 'CARLISLE': '21039', 'CARROLL': '21041', 'CARTER': '21043', 'CASEY': '21045', 'CHRISTIAN': '21047', 'CLARK': '21049', 'CLAY': '21051', 'CLINTON': '21053', 'CRITTENDEN': '21055', 'CUMBERLAND': '21057', 'DAVIESS': '21059', 'EDMONSON': '21061', 'ELLIOTT': '21063', 'ESTILL': '21065', 'FAYETTE': '21067', 'FLEMING': '21069', 'FLOYD': '21071', 'FRANKLIN': '21073', 'FULTON': '21075', 'GALLATIN': '21077', 'GARRARD': '21079', 'GRANT': '21081', 'GRAVES': '21083', 'GRAYSON': '21085', 'GREEN': '21087', 'GREENUP': '21089', 'HANCOCK': '21091', 'HARDIN': '21093', 'HARLAN': '21095', 'HARRISON': '21097', 'HART': '21099', 'HENDERSON': '21101', 'HENRY': '21103', 'HICKMAN': '21105', 'HOPKINS': '21107', 'JACKSON': '21109', 'JEFFERSON': '21111', 'JESSAMINE': '21113', 'JOHNSON': '21115', 'KENTON': '21117', 'KNOTT': '21119', 'KNOX': '21121', 'LARUE': '21123', 'LAUREL': '21125', 'LAWRENCE': '21127', 'LEE': '21129', 'LESLIE': '21131', 'LETCHER': '21133', 'LEWIS': '21135', 'LINCOLN': '21137', 'LIVINGSTON': '21139', 'LOGAN': '21141', 'LYON': '21143', 'MCCRACKEN': '21145', 'MCCREARY': '21147', 'MCLEAN': '21149', 'MADISON': '21151', 'MAGOFFIN': '21153', 'MARION': '21155', 'MARSHALL': '21157', 'MARTIN': '21159', 'MASON': '21161', 'MEADE': '21163', 'MENIFEE': '21165', 'MERCER': '21167', 'METCALFE': '21169', 'MONROE': '21171', 'MONTGOMERY': '21173', 'MORGAN': '21175', 'MUHLENBERG': '21177', 'NELSON': '21179', 'NICHOLAS': '21181', 'OHIO': '21183', 'OLDHAM': '21185', 'OWEN': '21187', 'OWSLEY': '21189', 'PENDLETON': '21191', 'PERRY': '21193', 'PIKE': '21195', 'POWELL': '21197', 'PULASKI': '21199', 'ROBERTSON': '21201', 'ROCKCASTLE': '21203', 'ROWAN': '21205', 'RUSSELL': '21207', 'SCOTT': '21209', 'SHELBY': '21211', 'SIMPSON': '21213', 'SPENCER': '21215', 'TAYLOR': '21217', 'TODD': '21219', 'TRIGG': '21221', 'TRIMBLE': '21223', 'UNION': '21225', 'WARREN': '21227', 'WASHINGTON': '21229', 'WAYNE': '21231', 'WEBSTER': '21233', 'WHITLEY': '21235', 'WOLFE': '21237', 'WOODFORD': '21239'}, 'LA': {'ACADIA': '22001', 'ALLEN': '22003', 'ASCENSION': '22005', 'ASSUMPTION': '22007', 'AVOYELLES': '22009', 'BEAUREGARD': '22011', 'BIENVILLE': '22013', 'BOSSIER': '22015', 'CADDO': '22017', 'CALCASIEU': '22019', 'CALDWELL': '22021', 'CAMERON': '22023', 'CATAHOULA': '22025', 'CLAIBORNE': '22027', 'CONCORDIA': '22029', 'DE SOTO': '22031', 'EAST BATON ROUGE': '22033', 'EAST CARROLL': '22035', 'EAST FELICIANA': '22037', 'EVANGELINE': '22039', 'FRANKLIN': '22041', 'GRANT': '22043', 'IBERIA': '22045', 'IBERVILLE': '22047', 'JACKSON': '22049', 'JEFFERSON': '22051', 'JEFFERSON DAVIS': '22053', 'LAFAYETTE': '22055', 'LAFOURCHE': '22057', 'LA SALLE': '22059', 'LINCOLN': '22061', 'LIVINGSTON': '22063', 'MADISON': '22065', 'MOREHOUSE': '22067', 'NATCHITOCHES': '22069', 'ORLEANS': '22071', 'OUACHITA': '22073', 'PLAQUEMINES': '22075', 'POINTE COUPEE': '22077', 'RAPIDES': '22079', 'RED RIVER': '22081', 'RICHLAND': '22083', 'SABINE': '22085', 'ST BERNARD': '22087', 'ST CHARLES': '22089', 'ST HELENA': '22091', 'ST JAMES': '22093', 'ST JOHN THE BAPTIST': '22095', 'ST LANDRY': '22097', 'ST MARTIN': '22099', 'ST MARY': '22101', 'ST TAMMANY': '22103', 'TANGIPAHOA': '22105', 'TENSAS': '22107', 'TERREBONNE': '22109', 'UNION': '22111', 'VERMILION': '22113', 'VERNON': '22115', 'WASHINGTON': '22117', 'WEBSTER': '22119', 'WEST BATON ROUGE': '22121', 'WEST CARROLL': '22123', 'WEST FELICIANA': '22125', 'WINN': '22127'}, 'ME': {'ANDROSCOGGIN': '23001', 'AROOSTOOK': '23003', 'CUMBERLAND': '23005', 'FRANKLIN': '23007', 'HANCOCK': '23009', 'KENNEBEC': '23011', 'KNOX': '23013', 'LINCOLN': '23015', 'OXFORD': '23017', 'PENOBSCOT': '23019', 'PISCATAQUIS': '23021', 'SAGADAHOC': '23023', 'SOMERSET': '23025', 'WALDO': '23027', 'WASHINGTON': '23029', 'YORK': '23031'}, 'MD': {'ALLEGANY': '24001', 'ANNE ARUNDEL': '24003', 'BALTIMORE': '24005', 'CALVERT': '24009', 'CAROLINE': '24011', 'CARROLL': '24013', 'CECIL': '24015', 'CHARLES': '24017', 'DORCHESTER': '24019', 'FREDERICK': '24021', 'GARRETT': '24023', 'HARFORD': '24025', 'HOWARD': '24027', 'KENT': '24029', 'MONTGOMERY': '24031', 'PRINCE GEORGES': '24033', 'QUEEN ANNES': '24035', 'ST MARYS': '24037', 'SOMERSET': '24039', 'TALBOT': '24041', 'WASHINGTON': '24043', 'WICOMICO': '24045', 'WORCESTER': '24047', 'BALTIMORE CITY': '24510'}, 'MA': {'BARNSTABLE': '25001', 'BERKSHIRE': '25003', 'BRISTOL': '25005', 'DUKES': '25007', 'ESSEX': '25009', 'FRANKLIN': '25011', 'HAMPDEN': '25013', 'HAMPSHIRE': '25015', 'MIDDLESEX': '25017', 'NANTUCKET': '25019', 'NORFOLK': '25021', 'PLYMOUTH': '25023', 'SUFFOLK': '25025', 'WORCESTER': '25027'}, 'MI': {'ALCONA': '26001', 'ALGER': '26003', 'ALLEGAN': '26005', 'ALPENA': '26007', 'ANTRIM': '26009', 'ARENAC': '26011', 'BARAGA': '26013', 'BARRY': '26015', 'BAY': '26017', 'BENZIE': '26019', 'BERRIEN': '26021', 'BRANCH': '26023', 'CALHOUN': '26025', 'CASS': '26027', 'CHARLEVOIX': '26029', 'CHEBOYGAN': '26031', 'CHIPPEWA': '26033', 'CLARE': '26035', 'CLINTON': '26037', 'CRAWFORD': '26039', 'DELTA': '26041', 'DICKINSON': '26043', 'EATON': '26045', 'EMMET': '26047', 'GENESEE': '26049', 'GLADWIN': '26051', 'GOGEBIC': '26053', 'GRAND TRAVERSE': '26055', 'GRATIOT': '26057', 'HILLSDALE': '26059', 'HOUGHTON': '26061', 'HURON': '26063', 'INGHAM': '26065', 'IONIA': '26067', 'IOSCO': '26069', 'IRON': '26071', 'ISABELLA': '26073', 'JACKSON': '26075', 'KALAMAZOO': '26077', 'KALKASKA': '26079', 'KENT': '26081', 'KEWEENAW': '26083', 'LAKE': '26085', 'LAPEER': '26087', 'LEELANAU': '26089', 'LENAWEE': '26091', 'LIVINGSTON': '26093', 'LUCE': '26095', 'MACKINAC': '26097', 'MACOMB': '26099', 'MANISTEE': '26101', 'MARQUETTE': '26103', 'MASON': '26105', 'MECOSTA': '26107', 'MENOMINEE': '26109', 'MIDLAND': '26111', 'MISSAUKEE': '26113', 'MONROE': '26115', 'MONTCALM': '26117', 'MONTMORENCY': '26119', 'MUSKEGON': '26121', 'NEWAYGO': '26123', 'OAKLAND': '26125', 'OCEANA': '26127', 'OGEMAW': '26129', 'ONTONAGON': '26131', 'OSCEOLA': '26133', 'OSCODA': '26135', 'OTSEGO': '26137', 'OTTAWA': '26139', 'PRESQUE ISLE': '26141', 'ROSCOMMON': '26143', 'SAGINAW': '26145', 'ST CLAIR': '26147', 'ST JOSEPH': '26149', 'SANILAC': '26151', 'SCHOOLCRAFT': '26153', 'SHIAWASSEE': '26155', 'TUSCOLA': '26157', 'VAN BUREN': '26159', 'WASHTENAW': '26161', 'WAYNE': '26163', 'WEXFORD': '26165'}, 'MN': {'AITKIN': '27001', 'ANOKA': '27003', 'BECKER': '27005', 'BELTRAMI': '27007', 'BENTON': '27009', 'BIG STONE': '27011', 'BLUE EARTH': '27013', 'BROWN': '27015', 'CARLTON': '27017', 'CARVER': '27019', 'CASS': '27021', 'CHIPPEWA': '27023', 'CHISAGO': '27025', 'CLAY': '27027', 'CLEARWATER': '27029', 'COOK': '27031', 'COTTONWOOD': '27033', 'CROW WING': '27035', 'DAKOTA': '27037', 'DODGE': '27039', 'DOUGLAS': '27041', 'FARIBAULT': '27043', 'FILLMORE': '27045', 'FREEBORN': '27047', 'GOODHUE': '27049', 'GRANT': '27051', 'HENNEPIN': '27053', 'HOUSTON': '27055', 'HUBBARD': '27057', 'ISANTI': '27059', 'ITASCA': '27061', 'JACKSON': '27063', 'KANABEC': '27065', 'KANDIYOHI': '27067', 'KITTSON': '27069', 'KOOCHICHING': '27071', 'LAC QUI PARLE': '27073', 'LAKE': '27075', 'LAKE OF THE WOODS': '27077', 'LE SUEUR': '27079', 'LINCOLN': '27081', 'LYON': '27083', 'MCLEOD': '27085', 'MAHNOMEN': '27087', 'MARSHALL': '27089', 'MARTIN': '27091', 'MEEKER': '27093', 'MILLE LACS': '27095', 'MORRISON': '27097', 'MOWER': '27099', 'MURRAY': '27101', 'NICOLLET': '27103', 'NOBLES': '27105', 'NORMAN': '27107', 'OLMSTED': '27109', 'OTTER TAIL': '27111', 'PENNINGTON': '27113', 'PINE': '27115', 'PIPESTONE': '27117', 'POLK': '27119', 'POPE': '27121', 'RAMSEY': '27123', 'RED LAKE': '27125', 'REDWOOD': '27127', 'RENVILLE': '27129', 'RICE': '27131', 'ROCK': '27133', 'ROSEAU': '27135', 'ST LOUIS': '27137', 'SCOTT': '27139', 'SHERBURNE': '27141', 'SIBLEY': '27143', 'STEARNS': '27145', 'STEELE': '27147', 'STEVENS': '27149', 'SWIFT': '27151', 'TODD': '27153', 'TRAVERSE': '27155', 'WABASHA': '27157', 'WADENA': '27159', 'WASECA': '27161', 'WASHINGTON': '27163', 'WATONWAN': '27165', 'WILKIN': '27167', 'WINONA': '27169', 'WRIGHT': '27171', 'YELLOW MEDICINE': '27173'}, 'MS': {'ADAMS': '28001', 'ALCORN': '28003', 'AMITE': '28005', 'ATTALA': '28007', 'BENTON': '28009', 'BOLIVAR': '28011', 'CALHOUN': '28013', 'CARROLL': '28015', 'CHICKASAW': '28017', 'CHOCTAW': '28019', 'CLAIBORNE': '28021', 'CLARKE': '28023', 'CLAY': '28025', 'COAHOMA': '28027', 'COPIAH': '28029', 'COVINGTON': '28031', 'DE SOTO': '28033', 'FORREST': '28035', 'FRANKLIN': '28037', 'GEORGE': '28039', 'GREENE': '28041', 'GRENADA': '28043', 'HANCOCK': '28045', 'HARRISON': '28047', 'HINDS': '28049', 'HOLMES': '28051', 'HUMPHREYS': '28053', 'ISSAQUENA': '28055', 'ITAWAMBA': '28057', 'JACKSON': '28059', 'JASPER': '28061', 'JEFFERSON': '28063', 'JEFFERSON DAVIS': '28065', 'JONES': '28067', 'KEMPER': '28069', 'LAFAYETTE': '28071', 'LAMAR': '28073', 'LAUDERDALE': '28075', 'LAWRENCE': '28077', 'LEAKE': '28079', 'LEE': '28081', 'LEFLORE': '28083', 'LINCOLN': '28085', 'LOWNDES': '28087', 'MADISON': '28089', 'MARION': '28091', 'MARSHALL': '28093', 'MONROE': '28095', 'MONTGOMERY': '28097', 'NESHOBA': '28099', 'NEWTON': '28101', 'NOXUBEE': '28103', 'OKTIBBEHA': '28105', 'PANOLA': '28107', 'PEARL RIVER': '28109', 'PERRY': '28111', 'PIKE': '28113', 'PONTOTOC': '28115', 'PRENTISS': '28117', 'QUITMAN': '28119', 'RANKIN': '28121', 'SCOTT': '28123', 'SHARKEY': '28125', 'SIMPSON': '28127', 'SMITH': '28129', 'STONE': '28131', 'SUNFLOWER': '28133', 'TALLAHATCHIE': '28135', 'TATE': '28137', 'TIPPAH': '28139', 'TISHOMINGO': '28141', 'TUNICA': '28143', 'UNION': '28145', 'WALTHALL': '28147', 'WARREN': '28149', 'WASHINGTON': '28151', 'WAYNE': '28153', 'WEBSTER': '28155', 'WILKINSON': '28157', 'WINSTON': '28159', 'YALOBUSHA': '28161', 'YAZOO': '28163'}, 'MO': {'ADAIR': '29001', 'ANDREW': '29003', 'ATCHISON': '29005', 'AUDRAIN': '29007', 'BARRY': '29009', 'BARTON': '29011', 'BATES': '29013', 'BENTON': '29015', 'BOLLINGER': '29017', 'BOONE': '29019', 'BUCHANAN': '29021', 'BUTLER': '29023', 'CALDWELL': '29025', 'CALLAWAY': '29027', 'CAMDEN': '29029', 'CAPE GIRARDEAU': '29031', 'CARROLL': '29033', 'CARTER': '29035', 'CASS': '29037', 'CEDAR': '29039', 'CHARITON': '29041', 'CHRISTIAN': '29043', 'CLARK': '29045', 'CLAY': '29047', 'CLINTON': '29049', 'COLE': '29051', 'COOPER': '29053', 'CRAWFORD': '29055', 'DADE': '29057', 'DALLAS': '29059', 'DAVIESS': '29061', 'DE KALB': '29063', 'DENT': '29065', 'DOUGLAS': '29067', 'DUNKLIN': '29069', 'FRANKLIN': '29071', 'GASCONADE': '29073', 'GENTRY': '29075', 'GREENE': '29077', 'GRUNDY': '29079', 'HARRISON': '29081', 'HENRY': '29083', 'HICKORY': '29085', 'HOLT': '29087', 'HOWARD': '29089', 'HOWELL': '29091', 'IRON': '29093', 'JACKSON': '29095', 'JASPER': '29097', 'JEFFERSON': '29099', 'JOHNSON': '29101', 'KNOX': '29103', 'LACLEDE': '29105', 'LAFAYETTE': '29107', 'LAWRENCE': '29109', 'LEWIS': '29111', 'LINCOLN': '29113', 'LINN': '29115', 'LIVINGSTON': '29117', 'MCDONALD': '29119', 'MACON': '29121', 'MADISON': '29123', 'MARIES': '29125', 'MARION': '29127', 'MERCER': '29129', 'MILLER': '29131', 'MISSISSIPPI': '29133', 'MONITEAU': '29135', 'MONROE': '29137', 'MONTGOMERY': '29139', 'MORGAN': '29141', 'NEW MADRID': '29143', 'NEWTON': '29145', 'NODAWAY': '29147', 'OREGON': '29149', 'OSAGE': '29151', 'OZARK': '29153', 'PEMISCOT': '29155', 'PERRY': '29157', 'PETTIS': '29159', 'PHELPS': '29161', 'PIKE': '29163', 'PLATTE': '29165', 'POLK': '29167', 'PULASKI': '29169', 'PUTNAM': '29171', 'RALLS': '29173', 'RANDOLPH': '29175', 'RAY': '29177', 'REYNOLDS': '29179', 'RIPLEY': '29181', 'ST CHARLES': '29183', 'ST CLAIR': '29185', 'STE. GENEVIEVE': '29186', 'ST FRANCOIS': '29187', 'ST LOUIS': '29189', 'SALINE': '29195', 'SCHUYLER': '29197', 'SCOTLAND': '29199', 'SCOTT': '29201', 'SHANNON': '29203', 'SHELBY': '29205', 'STODDARD': '29207', 'STONE': '29209', 'SULLIVAN': '29211', 'TANEY': '29213', 'TEXAS': '29215', 'VERNON': '29217', 'WARREN': '29219', 'WASHINGTON': '29221', 'WAYNE': '29223', 'WEBSTER': '29225', 'WORTH': '29227', 'WRIGHT': '29229', 'ST LOUIS CITY': '29510'}, 'MT': {'BEAVERHEAD': '30001', 'BIG HORN': '30003', 'BLAINE': '30005', 'BROADWATER': '30007', 'CARBON': '30009', 'CARTER': '30011', 'CASCADE': '30013', 'CHOUTEAU': '30015', 'CUSTER': '30017', 'DANIELS': '30019', 'DAWSON': '30021', 'DEER LODGE': '30023', 'FALLON': '30025', 'FERGUS': '30027', 'FLATHEAD': '30029', 'GALLATIN': '30031', 'GARFIELD': '30033', 'GLACIER': '30035', 'GOLDEN VALLEY': '30037', 'GRANITE': '30039', 'HILL': '30041', 'JEFFERSON': '30043', 'JUDITH BASIN': '30045', 'LAKE': '30047', 'LEWIS AND CLARK': '30049', 'LIBERTY': '30051', 'LINCOLN': '30053', 'MCCONE': '30055', 'MADISON': '30057', 'MEAGHER': '30059', 'MINERAL': '30061', 'MISSOULA': '30063', 'MUSSELSHELL': '30065', 'PARK': '30067', 'PETROLEUM': '30069', 'PHILLIPS': '30071', 'PONDERA': '30073', 'POWDER RIVER': '30075', 'POWELL': '30077', 'PRAIRIE': '30079', 'RAVALLI': '30081', 'RICHLAND': '30083', 'ROOSEVELT': '30085', 'ROSEBUD': '30087', 'SANDERS': '30089', 'SHERIDAN': '30091', 'SILVER BOW': '30093', 'STILLWATER': '30095', 'SWEET GRASS': '30097', 'TETON': '30099', 'TOOLE': '30101', 'TREASURE': '30103', 'VALLEY': '30105', 'WHEATLAND': '30107', 'WIBAUX': '30109', 'YELLOWSTONE': '30111', 'YELLOWSTONE NAT PARK': '30113'}, 'NE': {'ADAMS': '31001', 'ANTELOPE': '31003', 'ARTHUR': '31005', 'BANNER': '31007', 'BLAINE': '31009', 'BOONE': '31011', 'BOX BUTTE': '31013', 'BOYD': '31015', 'BROWN': '31017', 'BUFFALO': '31019', 'BURT': '31021', 'BUTLER': '31023', 'CASS': '31025', 'CEDAR': '31027', 'CHASE': '31029', 'CHERRY': '31031', 'CHEYENNE': '31033', 'CLAY': '31035', 'COLFAX': '31037', 'CUMING': '31039', 'CUSTER': '31041', 'DAKOTA': '31043', 'DAWES': '31045', 'DAWSON': '31047', 'DEUEL': '31049', 'DIXON': '31051', 'DODGE': '31053', 'DOUGLAS': '31055', 'DUNDY': '31057', 'FILLMORE': '31059', 'FRANKLIN': '31061', 'FRONTIER': '31063', 'FURNAS': '31065', 'GAGE': '31067', 'GARDEN': '31069', 'GARFIELD': '31071', 'GOSPER': '31073', 'GRANT': '31075', 'GREELEY': '31077', 'HALL': '31079', 'HAMILTON': '31081', 'HARLAN': '31083', 'HAYES': '31085', 'HITCHCOCK': '31087', 'HOLT': '31089', 'HOOKER': '31091', 'HOWARD': '31093', 'JEFFERSON': '31095', 'JOHNSON': '31097', 'KEARNEY': '31099', 'KEITH': '31101', 'KEYA PAHA': '31103', 'KIMBALL': '31105', 'KNOX': '31107', 'LANCASTER': '31109', 'LINCOLN': '31111', 'LOGAN': '31113', 'LOUP': '31115', 'MCPHERSON': '31117', 'MADISON': '31119', 'MERRICK': '31121', 'MORRILL': '31123', 'NANCE': '31125', 'NEMAHA': '31127', 'NUCKOLLS': '31129', 'OTOE': '31131', 'PAWNEE': '31133', 'PERKINS': '31135', 'PHELPS': '31137', 'PIERCE': '31139', 'PLATTE': '31141', 'POLK': '31143', 'RED WILLOW': '31145', 'RICHARDSON': '31147', 'ROCK': '31149', 'SALINE': '31151', 'SARPY': '31153', 'SAUNDERS': '31155', 'SCOTTS BLUFF': '31157', 'SEWARD': '31159', 'SHERIDAN': '31161', 'SHERMAN': '31163', 'SIOUX': '31165', 'STANTON': '31167', 'THAYER': '31169', 'THOMAS': '31171', 'THURSTON': '31173', 'VALLEY': '31175', 'WASHINGTON': '31177', 'WAYNE': '31179', 'WEBSTER': '31181', 'WHEELER': '31183', 'YORK': '31185'}, 'NV': {'CHURCHILL': '32001', 'CLARK': '32003', 'DOUGLAS': '32005', 'ELKO': '32007', 'ESMERALDA': '32009', 'EUREKA': '32011', 'HUMBOLDT': '32013', 'LANDER': '32015', 'LINCOLN': '32017', 'LYON': '32019', 'MINERAL': '32021', 'NYE': '32023', 'PERSHING': '32027', 'STOREY': '32029', 'WASHOE': '32031', 'WHITE PINE': '32033', 'CARSON CITY': '32510'}, 'NH': {'BELKNAP': '33001', 'CARROLL': '33003', 'CHESHIRE': '33005', 'COOS': '33007', 'GRAFTON': '33009', 'HILLSBOROUGH': '33011', 'MERRIMACK': '33013', 'ROCKINGHAM': '33015', 'STRAFFORD': '33017', 'SULLIVAN': '33019'}, 'NJ': {'ATLANTIC': '34001', 'BERGEN': '34003', 'BURLINGTON': '34005', 'CAMDEN': '34007', 'CAPE MAY': '34009', 'CUMBERLAND': '34011', 'ESSEX': '34013', 'GLOUCESTER': '34015', 'HUDSON': '34017', 'HUNTERDON': '34019', 'MERCER': '34021', 'MIDDLESEX': '34023', 'MONMOUTH': '34025', 'MORRIS': '34027', 'OCEAN': '34029', 'PASSAIC': '34031', 'SALEM': '34033', 'SOMERSET': '34035', 'SUSSEX': '34037', 'UNION': '34039', 'WARREN': '34041'}, 'NM': {'BERNALILLO': '35001', 'CATRON': '35003', 'CHAVES': '35005', 'CIBOLA': '35006', 'COLFAX': '35007', 'CURRY': '35009', 'DE BACA': '35011', 'DONA ANA': '35013', 'EDDY': '35015', 'GRANT': '35017', 'GUADALUPE': '35019', 'HARDING': '35021', 'HIDALGO': '35023', 'LEA': '35025', 'LINCOLN': '35027', 'LOS ALAMOS': '35028', 'LUNA': '35029', 'MCKINLEY': '35031', 'MORA': '35033', 'OTERO': '35035', 'QUAY': '35037', 'RIO ARRIBA': '35039', 'ROOSEVELT': '35041', 'SANDOVAL': '35043', 'SAN JUAN': '35045', 'SAN MIGUEL': '35047', 'SANTA FE': '35049', 'SIERRA': '35051', 'SOCORRO': '35053', 'TAOS': '35055', 'TORRANCE': '35057', 'UNION': '35059', 'VALENCIA': '35061'}, 'NY': {'ALBANY': '36001', 'ALLEGANY': '36003', 'BRONX': '36005', 'BROOME': '36007', 'CATTARAUGUS': '36009', 'CAYUGA': '36011', 'CHAUTAUQUA': '36013', 'CHEMUNG': '36015', 'CHENANGO': '36017', 'CLINTON': '36019', 'COLUMBIA': '36021', 'CORTLAND': '36023', 'DELAWARE': '36025', 'DUTCHESS': '36027', 'ERIE': '36029', 'ESSEX': '36031', 'FRANKLIN': '36033', 'FULTON': '36035', 'GENESEE': '36037', 'GREENE': '36039', 'HAMILTON': '36041', 'HERKIMER': '36043', 'JEFFERSON': '36045', 'KINGS': '36047', 'LEWIS': '36049', 'LIVINGSTON': '36051', 'MADISON': '36053', 'MONROE': '36055', 'MONTGOMERY': '36057', 'NASSAU': '36059', 'NEW YORK': '36061', 'NIAGARA': '36063', 'ONEIDA': '36065', 'ONONDAGA': '36067', 'ONTARIO': '36069', 'ORANGE': '36071', 'ORLEANS': '36073', 'OSWEGO': '36075', 'OTSEGO': '36077', 'PUTNAM': '36079', 'QUEENS': '36081', 'RENSSELAER': '36083', 'RICHMOND': '36085', 'ROCKLAND': '36087', 'ST LAWRENCE': '36089', 'SARATOGA': '36091', 'SCHENECTADY': '36093', 'SCHOHARIE': '36095', 'SCHUYLER': '36097', 'SENECA': '36099', 'STEUBEN': '36101', 'SUFFOLK': '36103', 'SULLIVAN': '36105', 'TIOGA': '36107', 'TOMPKINS': '36109', 'ULSTER': '36111', 'WARREN': '36113', 'WASHINGTON': '36115', 'WAYNE': '36117', 'WESTCHESTER': '36119', 'WYOMING': '36121', 'YATES': '36123'}, 'NC': {'ALAMANCE': '37001', 'ALEXANDER': '37003', 'ALLEGHANY': '37005', 'ANSON': '37007', 'ASHE': '37009', 'AVERY': '37011', 'BEAUFORT': '37013', 'BERTIE': '37015', 'BLADEN': '37017', 'BRUNSWICK': '37019', 'BUNCOMBE': '37021', 'BURKE': '37023', 'CABARRUS': '37025', 'CALDWELL': '37027', 'CAMDEN': '37029', 'CARTERET': '37031', 'CASWELL': '37033', 'CATAWBA': '37035', 'CHATHAM': '37037', 'CHEROKEE': '37039', 'CHOWAN': '37041', 'CLAY': '37043', 'CLEVELAND': '37045', 'COLUMBUS': '37047', 'CRAVEN': '37049', 'CUMBERLAND': '37051', 'CURRITUCK': '37053', 'DARE': '37055', 'DAVIDSON': '37057', 'DAVIE': '37059', 'DUPLIN': '37061', 'DURHAM': '37063', 'EDGECOMBE': '37065', 'FORSYTH': '37067', 'FRANKLIN': '37069', 'GASTON': '37071', 'GATES': '37073', 'GRAHAM': '37075', 'GRANVILLE': '37077', 'GREENE': '37079', 'GUILFORD': '37081', 'HALIFAX': '37083', 'HARNETT': '37085', 'HAYWOOD': '37087', 'HENDERSON': '37089', 'HERTFORD': '37091', 'HOKE': '37093', 'HYDE': '37095', 'IREDELL': '37097', 'JACKSON': '37099', 'JOHNSTON': '37101', 'JONES': '37103', 'LEE': '37105', 'LENOIR': '37107', 'LINCOLN': '37109', 'MCDOWELL': '37111', 'MACON': '37113', 'MADISON': '37115', 'MARTIN': '37117', 'MECKLENBURG': '37119', 'MITCHELL': '37121', 'MONTGOMERY': '37123', 'MOORE': '37125', 'NASH': '37127', 'NEW HANOVER': '37129', 'NORTHAMPTON': '37131', 'ONSLOW': '37133', 'ORANGE': '37135', 'PAMLICO': '37137', 'PASQUOTANK': '37139', 'PENDER': '37141', 'PERQUIMANS': '37143', 'PERSON': '37145', 'PITT': '37147', 'POLK': '37149', 'RANDOLPH': '37151', 'RICHMOND': '37153', 'ROBESON': '37155', 'ROCKINGHAM': '37157', 'ROWAN': '37159', 'RUTHERFORD': '37161', 'SAMPSON': '37163', 'SCOTLAND': '37165', 'STANLY': '37167', 'STOKES': '37169', 'SURRY': '37171', 'SWAIN': '37173', 'TRANSYLVANIA': '37175', 'TYRRELL': '37177', 'UNION': '37179', 'VANCE': '37181', 'WAKE': '37183', 'WARREN': '37185', 'WASHINGTON': '37187', 'WATAUGA': '37189', 'WAYNE': '37191', 'WILKES': '37193', 'WILSON': '37195', 'YADKIN': '37197', 'YANCEY': '37199'}, 'ND': {'ADAMS': '38001', 'BARNES': '38003', 'BENSON': '38005', 'BILLINGS': '38007', 'BOTTINEAU': '38009', 'BOWMAN': '38011', 'BURKE': '38013', 'BURLEIGH': '38015', 'CASS': '38017', 'CAVALIER': '38019', 'DICKEY': '38021', 'DIVIDE': '38023', 'DUNN': '38025', 'EDDY': '38027', 'EMMONS': '38029', 'FOSTER': '38031', 'GOLDEN VALLEY': '38033', 'GRAND FORKS': '38035', 'GRANT': '38037', 'GRIGGS': '38039', 'HETTINGER': '38041', 'KIDDER': '38043', 'LA MOURE': '38045', 'LOGAN': '38047', 'MCHENRY': '38049', 'MCINTOSH': '38051', 'MCKENZIE': '38053', 'MCLEAN': '38055', 'MERCER': '38057', 'MORTON': '38059', 'MOUNTRAIL': '38061', 'NELSON': '38063', 'OLIVER': '38065', 'PEMBINA': '38067', 'PIERCE': '38069', 'RAMSEY': '38071', 'RANSOM': '38073', 'RENVILLE': '38075', 'RICHLAND': '38077', 'ROLETTE': '38079', 'SARGENT': '38081', 'SHERIDAN': '38083', 'SIOUX': '38085', 'SLOPE': '38087', 'STARK': '38089', 'STEELE': '38091', 'STUTSMAN': '38093', 'TOWNER': '38095', 'TRAILL': '38097', 'WALSH': '38099', 'WARD': '38101', 'WELLS': '38103', 'WILLIAMS': '38105'}, 'OH': {'ADAMS': '39001', 'ALLEN': '39003', 'ASHLAND': '39005', 'ASHTABULA': '39007', 'ATHENS': '39009', 'AUGLAIZE': '39011', 'BELMONT': '39013', 'BROWN': '39015', 'BUTLER': '39017', 'CARROLL': '39019', 'CHAMPAIGN': '39021', 'CLARK': '39023', 'CLERMONT': '39025', 'CLINTON': '39027', 'COLUMBIANA': '39029', 'COSHOCTON': '39031', 'CRAWFORD': '39033', 'CUYAHOGA': '39035', 'DARKE': '39037', 'DEFIANCE': '39039', 'DELAWARE': '39041', 'ERIE': '39043', 'FAIRFIELD': '39045', 'FAYETTE': '39047', 'FRANKLIN': '39049', 'FULTON': '39051', 'GALLIA': '39053', 'GEAUGA': '39055', 'GREENE': '39057', 'GUERNSEY': '39059', 'HAMILTON': '39061', 'HANCOCK': '39063', 'HARDIN': '39065', 'HARRISON': '39067', 'HENRY': '39069', 'HIGHLAND': '39071', 'HOCKING': '39073', 'HOLMES': '39075', 'HURON': '39077', 'JACKSON': '39079', 'JEFFERSON': '39081', 'KNOX': '39083', 'LAKE': '39085', 'LAWRENCE': '39087', 'LICKING': '39089', 'LOGAN': '39091', 'LORAIN': '39093', 'LUCAS': '39095', 'MADISON': '39097', 'MAHONING': '39099', 'MARION': '39101', 'MEDINA': '39103', 'MEIGS': '39105', 'MERCER': '39107', 'MIAMI': '39109', 'MONROE': '39111', 'MONTGOMERY': '39113', 'MORGAN': '39115', 'MORROW': '39117', 'MUSKINGUM': '39119', 'NOBLE': '39121', 'OTTAWA': '39123', 'PAULDING': '39125', 'PERRY': '39127', 'PICKAWAY': '39129', 'PIKE': '39131', 'PORTAGE': '39133', 'PREBLE': '39135', 'PUTNAM': '39137', 'RICHLAND': '39139', 'ROSS': '39141', 'SANDUSKY': '39143', 'SCIOTO': '39145', 'SENECA': '39147', 'SHELBY': '39149', 'STARK': '39151', 'SUMMIT': '39153', 'TRUMBULL': '39155', 'TUSCARAWAS': '39157', 'UNION': '39159', 'VAN WERT': '39161', 'VINTON': '39163', 'WARREN': '39165', 'WASHINGTON': '39167', 'WAYNE': '39169', 'WILLIAMS': '39171', 'WOOD': '39173', 'WYANDOT': '39175'}, 'OK': {'ADAIR': '40001', 'ALFALFA': '40003', 'ATOKA': '40005', 'BEAVER': '40007', 'BECKHAM': '40009', 'BLAINE': '40011', 'BRYAN': '40013', 'CADDO': '40015', 'CANADIAN': '40017', 'CARTER': '40019', 'CHEROKEE': '40021', 'CHOCTAW': '40023', 'CIMARRON': '40025', 'CLEVELAND': '40027', 'COAL': '40029', 'COMANCHE': '40031', 'COTTON': '40033', 'CRAIG': '40035', 'CREEK': '40037', 'CUSTER': '40039', 'DELAWARE': '40041', 'DEWEY': '40043', 'ELLIS': '40045', 'GARFIELD': '40047', 'GARVIN': '40049', 'GRADY': '40051', 'GRANT': '40053', 'GREER': '40055', 'HARMON': '40057', 'HARPER': '40059', 'HASKELL': '40061', 'HUGHES': '40063', 'JACKSON': '40065', 'JEFFERSON': '40067', 'JOHNSTON': '40069', 'KAY': '40071', 'KINGFISHER': '40073', 'KIOWA': '40075', 'LATIMER': '40077', 'LE FLORE': '40079', 'LINCOLN': '40081', 'LOGAN': '40083', 'LOVE': '40085', 'MCCLAIN': '40087', 'MCCURTAIN': '40089', 'MCINTOSH': '40091', 'MAJOR': '40093', 'MARSHALL': '40095', 'MAYES': '40097', 'MURRAY': '40099', 'MUSKOGEE': '40101', 'NOBLE': '40103', 'NOWATA': '40105', 'OKFUSKEE': '40107', 'OKLAHOMA': '40109', 'OKMULGEE': '40111', 'OSAGE': '40113', 'OTTAWA': '40115', 'PAWNEE': '40117', 'PAYNE': '40119', 'PITTSBURG': '40121', 'PONTOTOC': '40123', 'POTTAWATOMIE': '40125', 'PUSHMATAHA': '40127', 'ROGER MILLS': '40129', 'ROGERS': '40131', 'SEMINOLE': '40133', 'SEQUOYAH': '40135', 'STEPHENS': '40137', 'TEXAS': '40139', 'TILLMAN': '40141', 'TULSA': '40143', 'WAGONER': '40145', 'WASHINGTON': '40147', 'WASHITA': '40149', 'WOODS': '40151', 'WOODWARD': '40153'}, 'OR': {'BAKER': '41001', 'BENTON': '41003', 'CLACKAMAS': '41005', 'CLATSOP': '41007', 'COLUMBIA': '41009', 'COOS': '41011', 'CROOK': '41013', 'CURRY': '41015', 'DESCHUTES': '41017', 'DOUGLAS': '41019', 'GILLIAM': '41021', 'GRANT': '41023', 'HARNEY': '41025', 'HOOD RIVER': '41027', 'JACKSON': '41029', 'JEFFERSON': '41031', 'JOSEPHINE': '41033', 'KLAMATH': '41035', 'LAKE': '41037', 'LANE': '41039', 'LINCOLN': '41041', 'LINN': '41043', 'MALHEUR': '41045', 'MARION': '41047', 'MORROW': '41049', 'MULTNOMAH': '41051', 'POLK': '41053', 'SHERMAN': '41055', 'TILLAMOOK': '41057', 'UMATILLA': '41059', 'UNION': '41061', 'WALLOWA': '41063', 'WASCO': '41065', 'WASHINGTON': '41067', 'WHEELER': '41069', 'YAMHILL': '41071'}, 'PA': {'ADAMS': '42001', 'ALLEGHENY': '42003', 'ARMSTRONG': '42005', 'BEAVER': '42007', 'BEDFORD': '42009', 'BERKS': '42011', 'BLAIR': '42013', 'BRADFORD': '42015', 'BUCKS': '42017', 'BUTLER': '42019', 'CAMBRIA': '42021', 'CAMERON': '42023', 'CARBON': '42025', 'CENTRE': '42027', 'CHESTER': '42029', 'CLARION': '42031', 'CLEARFIELD': '42033', 'CLINTON': '42035', 'COLUMBIA': '42037', 'CRAWFORD': '42039', 'CUMBERLAND': '42041', 'DAUPHIN': '42043', 'DELAWARE': '42045', 'ELK': '42047', 'ERIE': '42049', 'FAYETTE': '42051', 'FOREST': '42053', 'FRANKLIN': '42055', 'FULTON': '42057', 'GREENE': '42059', 'HUNTINGDON': '42061', 'INDIANA': '42063', 'JEFFERSON': '42065', 'JUNIATA': '42067', 'LACKAWANNA': '42069', 'LANCASTER': '42071', 'LAWRENCE': '42073', 'LEBANON': '42075', 'LEHIGH': '42077', 'LUZERNE': '42079', 'LYCOMING': '42081', 'MCKEAN': '42083', 'MERCER': '42085', 'MIFFLIN': '42087', 'MONROE': '42089', 'MONTGOMERY': '42091', 'MONTOUR': '42093', 'NORTHAMPTON': '42095', 'NORTHUMBERLAND': '42097', 'PERRY': '42099', 'PHILADELPHIA': '42101', 'PIKE': '42103', 'POTTER': '42105', 'SCHUYLKILL': '42107', 'SNYDER': '42109', 'SOMERSET': '42111', 'SULLIVAN': '42113', 'SUSQUEHANNA': '42115', 'TIOGA': '42117', 'UNION': '42119', 'VENANGO': '42121', 'WARREN': '42123', 'WASHINGTON': '42125', 'WAYNE': '42127', 'WESTMORELAND': '42129', 'WYOMING': '42131', 'YORK': '42133'}, 'RI': {'BRISTOL': '44001', 'KENT': '44003', 'NEWPORT': '44005', 'PROVIDENCE': '44007', 'WASHINGTON': '44009'}, 'SC': {'ABBEVILLE': '45001', 'AIKEN': '45003', 'ALLENDALE': '45005', 'ANDERSON': '45007', 'BAMBERG': '45009', 'BARNWELL': '45011', 'BEAUFORT': '45013', 'BERKELEY': '45015', 'CALHOUN': '45017', 'CHARLESTON': '45019', 'CHEROKEE': '45021', 'CHESTER': '45023', 'CHESTERFIELD': '45025', 'CLARENDON': '45027', 'COLLETON': '45029', 'DARLINGTON': '45031', 'DILLON': '45033', 'DORCHESTER': '45035', 'EDGEFIELD': '45037', 'FAIRFIELD': '45039', 'FLORENCE': '45041', 'GEORGETOWN': '45043', 'GREENVILLE': '45045', 'GREENWOOD': '45047', 'HAMPTON': '45049', 'HORRY': '45051', 'JASPER': '45053', 'KERSHAW': '45055', 'LANCASTER': '45057', 'LAURENS': '45059', 'LEE': '45061', 'LEXINGTON': '45063', 'MCCORMICK': '45065', 'MARION': '45067', 'MARLBORO': '45069', 'NEWBERRY': '45071', 'OCONEE': '45073', 'ORANGEBURG': '45075', 'PICKENS': '45077', 'RICHLAND': '45079', 'SALUDA': '45081', 'SPARTANBURG': '45083', 'SUMTER': '45085', 'UNION': '45087', 'WILLIAMSBURG': '45089', 'YORK': '45091'}, 'SD': {'AURORA': '46003', 'BEADLE': '46005', 'BENNETT': '46007', 'BON HOMME': '46009', 'BROOKINGS': '46011', 'BROWN': '46013', 'BRULE': '46015', 'BUFFALO': '46017', 'BUTTE': '46019', 'CAMPBELL': '46021', 'CHARLES MIX': '46023', 'CLARK': '46025', 'CLAY': '46027', 'CODINGTON': '46029', 'CORSON': '46031', 'CUSTER': '46033', 'DAVISON': '46035', 'DAY': '46037', 'DEUEL': '46039', 'DEWEY': '46041', 'DOUGLAS': '46043', 'EDMUNDS': '46045', 'FALL RIVER': '46047', 'FAULK': '46049', 'GRANT': '46051', 'GREGORY': '46053', 'HAAKON': '46055', 'HAMLIN': '46057', 'HAND': '46059', 'HANSON': '46061', 'HARDING': '46063', 'HUGHES': '46065', 'HUTCHINSON': '46067', 'HYDE': '46069', 'JACKSON': '46071', 'JERAULD': '46073', 'JONES': '46075', 'KINGSBURY': '46077', 'LAKE': '46079', 'LAWRENCE': '46081', 'LINCOLN': '46083', 'LYMAN': '46085', 'MCCOOK': '46087', 'MCPHERSON': '46089', 'MARSHALL': '46091', 'MEADE': '46093', 'MELLETTE': '46095', 'MINER': '46097', 'MINNEHAHA': '46099', 'MOODY': '46101', 'PENNINGTON': '46103', 'PERKINS': '46105', 'POTTER': '46107', 'ROBERTS': '46109', 'SANBORN': '46111', 'SHANNON': '46113', 'SPINK': '46115', 'STANLEY': '46117', 'SULLY': '46119', 'TODD': '46121', 'TRIPP': '46123', 'TURNER': '46125', 'UNION': '46127', 'WALWORTH': '46129', 'YANKTON': '46135', 'ZIEBACH': '46137'}, 'TN': {'ANDERSON': '47001', 'BEDFORD': '47003', 'BENTON': '47005', 'BLEDSOE': '47007', 'BLOUNT': '47009', 'BRADLEY': '47011', 'CAMPBELL': '47013', 'CANNON': '47015', 'CARROLL': '47017', 'CARTER': '47019', 'CHEATHAM': '47021', 'CHESTER': '47023', 'CLAIBORNE': '47025', 'CLAY': '47027', 'COCKE': '47029', 'COFFEE': '47031', 'CROCKETT': '47033', 'CUMBERLAND': '47035', 'DAVIDSON': '47037', 'DECATUR': '47039', 'DE KALB': '47041', 'DICKSON': '47043', 'DYER': '47045', 'FAYETTE': '47047', 'FENTRESS': '47049', 'FRANKLIN': '47051', 'GIBSON': '47053', 'GILES': '47055', 'GRAINGER': '47057', 'GREENE': '47059', 'GRUNDY': '47061', 'HAMBLEN': '47063', 'HAMILTON': '47065', 'HANCOCK': '47067', 'HARDEMAN': '47069', 'HARDIN': '47071', 'HAWKINS': '47073', 'HAYWOOD': '47075', 'HENDERSON': '47077', 'HENRY': '47079', 'HICKMAN': '47081', 'HOUSTON': '47083', 'HUMPHREYS': '47085', 'JACKSON': '47087', 'JEFFERSON': '47089', 'JOHNSON': '47091', 'KNOX': '47093', 'LAKE': '47095', 'LAUDERDALE': '47097', 'LAWRENCE': '47099', 'LEWIS': '47101', 'LINCOLN': '47103', 'LOUDON': '47105', 'MCMINN': '47107', 'MCNAIRY': '47109', 'MACON': '47111', 'MADISON': '47113', 'MARION': '47115', 'MARSHALL': '47117', 'MAURY': '47119', 'MEIGS': '47121', 'MONROE': '47123', 'MONTGOMERY': '47125', 'MOORE': '47127', 'MORGAN': '47129', 'OBION': '47131', 'OVERTON': '47133', 'PERRY': '47135', 'PICKETT': '47137', 'POLK': '47139', 'PUTNAM': '47141', 'RHEA': '47143', 'ROANE': '47145', 'ROBERTSON': '47147', 'RUTHERFORD': '47149', 'SCOTT': '47151', 'SEQUATCHIE': '47153', 'SEVIER': '47155', 'SHELBY': '47157', 'SMITH': '47159', 'STEWART': '47161', 'SULLIVAN': '47163', 'SUMNER': '47165', 'TIPTON': '47167', 'TROUSDALE': '47169', 'UNICOI': '47171', 'UNION': '47173', 'VAN BUREN': '47175', 'WARREN': '47177', 'WASHINGTON': '47179', 'WAYNE': '47181', 'WEAKLEY': '47183', 'WHITE': '47185', 'WILLIAMSON': '47187', 'WILSON': '47189'}, 'TX': {'ANDERSON': '48001', 'ANDREWS': '48003', 'ANGELINA': '48005', 'ARANSAS': '48007', 'ARCHER': '48009', 'ARMSTRONG': '48011', 'ATASCOSA': '48013', 'AUSTIN': '48015', 'BAILEY': '48017', 'BANDERA': '48019', 'BASTROP': '48021', 'BAYLOR': '48023', 'BEE': '48025', 'BELL': '48027', 'BEXAR': '48029', 'BLANCO': '48031', 'BORDEN': '48033', 'BOSQUE': '48035', 'BOWIE': '48037', 'BRAZORIA': '48039', 'BRAZOS': '48041', 'BREWSTER': '48043', 'BRISCOE': '48045', 'BROOKS': '48047', 'BROWN': '48049', 'BURLESON': '48051', 'BURNET': '48053', 'CALDWELL': '48055', 'CALHOUN': '48057', 'CALLAHAN': '48059', 'CAMERON': '48061', 'CAMP': '48063', 'CARSON': '48065', 'CASS': '48067', 'CASTRO': '48069', 'CHAMBERS': '48071', 'CHEROKEE': '48073', 'CHILDRESS': '48075', 'CLAY': '48077', 'COCHRAN': '48079', 'COKE': '48081', 'COLEMAN': '48083', 'COLLIN': '48085', 'COLLINGSWORTH': '48087', 'COLORADO': '48089', 'COMAL': '48091', 'COMANCHE': '48093', 'CONCHO': '48095', 'COOKE': '48097', 'CORYELL': '48099', 'COTTLE': '48101', 'CRANE': '48103', 'CROCKETT': '48105', 'CROSBY': '48107', 'CULBERSON': '48109', 'DALLAM': '48111', 'DALLAS': '48113', 'DAWSON': '48115', 'DEAF SMITH': '48117', 'DELTA': '48119', 'DENTON': '48121', 'DE WITT': '48123', 'DICKENS': '48125', 'DIMMIT': '48127', 'DONLEY': '48129', 'DUVAL': '48131', 'EASTLAND': '48133', 'ECTOR': '48135', 'EDWARDS': '48137', 'ELLIS': '48139', 'EL PASO': '48141', 'ERATH': '48143', 'FALLS': '48145', 'FANNIN': '48147', 'FAYETTE': '48149', 'FISHER': '48151', 'FLOYD': '48153', 'FOARD': '48155', 'FORT BEND': '48157', 'FRANKLIN': '48159', 'FREESTONE': '48161', 'FRIO': '48163', 'GAINES': '48165', 'GALVESTON': '48167', 'GARZA': '48169', 'GILLESPIE': '48171', 'GLASSCOCK': '48173', 'GOLIAD': '48175', 'GONZALES': '48177', 'GRAY': '48179', 'GRAYSON': '48181', 'GREGG': '48183', 'GRIMES': '48185', 'GUADALUPE': '48187', 'HALE': '48189', 'HALL': '48191', 'HAMILTON': '48193', 'HANSFORD': '48195', 'HARDEMAN': '48197', 'HARDIN': '48199', 'HARRIS': '48201', 'HARRISON': '48203', 'HARTLEY': '48205', 'HASKELL': '48207', 'HAYS': '48209', 'HEMPHILL': '48211', 'HENDERSON': '48213', 'HIDALGO': '48215', 'HILL': '48217', 'HOCKLEY': '48219', 'HOOD': '48221', 'HOPKINS': '48223', 'HOUSTON': '48225', 'HOWARD': '48227', 'HUDSPETH': '48229', 'HUNT': '48231', 'HUTCHINSON': '48233', 'IRION': '48235', 'JACK': '48237', 'JACKSON': '48239', 'JASPER': '48241', 'JEFF DAVIS': '48243', 'JEFFERSON': '48245', 'JIM HOGG': '48247', 'JIM WELLS': '48249', 'JOHNSON': '48251', 'JONES': '48253', 'KARNES': '48255', 'KAUFMAN': '48257', 'KENDALL': '48259', 'KENEDY': '48261', 'KENT': '48263', 'KERR': '48265', 'KIMBLE': '48267', 'KING': '48269', 'KINNEY': '48271', 'KLEBERG': '48273', 'KNOX': '48275', 'LAMAR': '48277', 'LAMB': '48279', 'LAMPASAS': '48281', 'LA SALLE': '48283', 'LAVACA': '48285', 'LEE': '48287', 'LEON': '48289', 'LIBERTY': '48291', 'LIMESTONE': '48293', 'LIPSCOMB': '48295', 'LIVE OAK': '48297', 'LLANO': '48299', 'LOVING': '48301', 'LUBBOCK': '48303', 'LYNN': '48305', 'MCCULLOCH': '48307', 'MCLENNAN': '48309', 'MCMULLEN': '48311', 'MADISON': '48313', 'MARION': '48315', 'MARTIN': '48317', 'MASON': '48319', 'MATAGORDA': '48321', 'MAVERICK': '48323', 'MEDINA': '48325', 'MENARD': '48327', 'MIDLAND': '48329', 'MILAM': '48331', 'MILLS': '48333', 'MITCHELL': '48335', 'MONTAGUE': '48337', 'MONTGOMERY': '48339', 'MOORE': '48341', 'MORRIS': '48343', 'MOTLEY': '48345', 'NACOGDOCHES': '48347', 'NAVARRO': '48349', 'NEWTON': '48351', 'NOLAN': '48353', 'NUECES': '48355', 'OCHILTREE': '48357', 'OLDHAM': '48359', 'ORANGE': '48361', 'PALO PINTO': '48363', 'PANOLA': '48365', 'PARKER': '48367', 'PARMER': '48369', 'PECOS': '48371', 'POLK': '48373', 'POTTER': '48375', 'PRESIDIO': '48377', 'RAINS': '48379', 'RANDALL': '48381', 'REAGAN': '48383', 'REAL': '48385', 'RED RIVER': '48387', 'REEVES': '48389', 'REFUGIO': '48391', 'ROBERTS': '48393', 'ROBERTSON': '48395', 'ROCKWALL': '48397', 'RUNNELS': '48399', 'RUSK': '48401', 'SABINE': '48403', 'SAN AUGUSTINE': '48405', 'SAN JACINTO': '48407', 'SAN PATRICIO': '48409', 'SAN SABA': '48411', 'SCHLEICHER': '48413', 'SCURRY': '48415', 'SHACKELFORD': '48417', 'SHELBY': '48419', 'SHERMAN': '48421', 'SMITH': '48423', 'SOMERVELL': '48425', 'STARR': '48427', 'STEPHENS': '48429', 'STERLING': '48431', 'STONEWALL': '48433', 'SUTTON': '48435', 'SWISHER': '48437', 'TARRANT': '48439', 'TAYLOR': '48441', 'TERRELL': '48443', 'TERRY': '48445', 'THROCKMORTON': '48447', 'TITUS': '48449', 'TOM GREEN': '48451', 'TRAVIS': '48453', 'TRINITY': '48455', 'TYLER': '48457', 'UPSHUR': '48459', 'UPTON': '48461', 'UVALDE': '48463', 'VAL VERDE': '48465', 'VAN ZANDT': '48467', 'VICTORIA': '48469', 'WALKER': '48471', 'WALLER': '48473', 'WARD': '48475', 'WASHINGTON': '48477', 'WEBB': '48479', 'WHARTON': '48481', 'WHEELER': '48483', 'WICHITA': '48485', 'WILBARGER': '48487', 'WILLACY': '48489', 'WILLIAMSON': '48491', 'WILSON': '48493', 'WINKLER': '48495', 'WISE': '48497', 'WOOD': '48499', 'YOAKUM': '48501', 'YOUNG': '48503', 'ZAPATA': '48505', 'ZAVALA': '48507'}, 'UT': {'BEAVER': '49001', 'BOX ELDER': '49003', 'CACHE': '49005', 'CARBON': '49007', 'DAGGETT': '49009', 'DAVIS': '49011', 'DUCHESNE': '49013', 'EMERY': '49015', 'GARFIELD': '49017', 'GRAND': '49019', 'IRON': '49021', 'JUAB': '49023', 'KANE': '49025', 'MILLARD': '49027', 'MORGAN': '49029', 'PIUTE': '49031', 'RICH': '49033', 'SALT LAKE': '49035', 'SAN JUAN': '49037', 'SANPETE': '49039', 'SEVIER': '49041', 'SUMMIT': '49043', 'TOOELE': '49045', 'UINTAH': '49047', 'UTAH': '49049', 'WASATCH': '49051', 'WASHINGTON': '49053', 'WAYNE': '49055', 'WEBER': '49057'}, 'VT': {'ADDISON': '50001', 'BENNINGTON': '50003', 'CALEDONIA': '50005', 'CHITTENDEN': '50007', 'ESSEX': '50009', 'FRANKLIN': '50011', 'GRAND ISLE': '50013', 'LAMOILLE': '50015', 'ORANGE': '50017', 'ORLEANS': '50019', 'RUTLAND': '50021', 'WASHINGTON': '50023', 'WINDHAM': '50025', 'WINDSOR': '50027'}, 'VA': {'ACCOMACK': '51001', 'ALBEMARLE': '51003', 'ALLEGHANY': '51005', 'AMELIA': '51007', 'AMHERST': '51009', 'APPOMATTOX': '51011', 'ARLINGTON': '51013', 'AUGUSTA': '51015', 'BATH': '51017', 'BEDFORD': '51019', 'BLAND': '51021', 'BOTETOURT': '51023', 'BRUNSWICK': '51025', 'BUCHANAN': '51027', 'BUCKINGHAM': '51029', 'CAMPBELL': '51031', 'CAROLINE': '51033', 'CARROLL': '51035', 'CHARLES CITY': '51036', 'CHARLOTTE': '51037', 'CHESTERFIELD': '51041', 'CLARKE': '51043', 'CRAIG': '51045', 'CULPEPER': '51047', 'CUMBERLAND': '51049', 'DICKENSON': '51051', 'DINWIDDIE': '51053', 'ESSEX': '51057', 'FAIRFAX': '51059', 'FAUQUIER': '51061', 'FLOYD': '51063', 'FLUVANNA': '51065', 'FRANKLIN': '51067', 'FREDERICK': '51069', 'GILES': '51071', 'GLOUCESTER': '51073', 'GOOCHLAND': '51075', 'GRAYSON': '51077', 'GREENE': '51079', 'GREENSVILLE': '51081', 'HALIFAX': '51083', 'HANOVER': '51085', 'HENRICO': '51087', 'HENRY': '51089', 'HIGHLAND': '51091', 'ISLE OF WIGHT': '51093', 'JAMES CITY': '51095', 'KING AND QUEEN': '51097', 'KING GEORGE': '51099', 'KING WILLIAM': '51101', 'LANCASTER': '51103', 'LEE': '51105', 'LOUDOUN': '51107', 'LOUISA': '51109', 'LUNENBURG': '51111', 'MADISON': '51113', 'MATHEWS': '51115', 'MECKLENBURG': '51117', 'MIDDLESEX': '51119', 'MONTGOMERY': '51121', 'NELSON': '51125', 'NEW KENT': '51127', 'NORTHAMPTON': '51131', 'NORTHUMBERLAND': '51133', 'NOTTOWAY': '51135', 'ORANGE': '51137', 'PAGE': '51139', 'PATRICK': '51141', 'PITTSYLVANIA': '51143', 'POWHATAN': '51145', 'PRINCE EDWARD': '51147', 'PRINCE GEORGE': '51149', 'PRINCE WILLIAM': '51153', 'PULASKI': '51155', 'RAPPAHANNOCK': '51157', 'RICHMOND': '51159', 'ROANOKE': '51161', 'ROCKBRIDGE': '51163', 'ROCKINGHAM': '51165', 'RUSSELL': '51167', 'SCOTT': '51169', 'SHENANDOAH': '51171', 'SMYTH': '51173', 'SOUTHAMPTON': '51175', 'SPOTSYLVANIA': '51177', 'STAFFORD': '51179', 'SURRY': '51181', 'SUSSEX': '51183', 'TAZEWELL': '51185', 'WARREN': '51187', 'WASHINGTON': '51191', 'WESTMORELAND': '51193', 'WISE': '51195', 'WYTHE': '51197', 'YORK': '51199', 'ALEXANDRIA CITY': '51510', 'BEDFORD CITY': '51515', 'BRISTOL CITY': '51520', 'BUENA VISTA CITY': '51530', 'CHARLOTTESVILLE CITY': '51540', 'CHESAPEAKE CITY': '51550', 'CLIFTON FORGE CITY': '51560', 'COLONIAL HEIGHTS CIT': '51570', 'COVINGTON CITY': '51580', 'DANVILLE CITY': '51590', 'EMPORIA CITY': '51595', 'FAIRFAX CITY': '51600', 'FALLS CHURCH CITY': '51610', 'FRANKLIN CITY': '51620', 'FREDERICKSBURG CITY': '51630', 'GALAX CITY': '51640', 'HAMPTON CITY': '51650', 'HARRISONBURG CITY': '51660', 'HOPEWELL CITY': '51670', 'LEXINGTON CITY': '51678', 'LYNCHBURG CITY': '51680', 'MANASSAS CITY': '51683', 'MANASSAS PARK CITY': '51685', 'MARTINSVILLE CITY': '51690', 'NEWPORT NEWS CITY': '51700', 'NORFOLK CITY': '51710', 'NORTON CITY': '51720', 'PETERSBURG CITY': '51730', 'POQUOSON CITY': '51735', 'PORTSMOUTH CITY': '51740', 'RADFORD CITY': '51750', 'RICHMOND CITY': '51760', 'ROANOKE CITY': '51770', 'SALEM CITY': '51775', 'SOUTH BOSTON CITY': '51780', 'STAUNTON CITY': '51790', 'SUFFOLK CITY': '51800', 'VIRGINIA BEACH CITY': '51810', 'WAYNESBORO CITY': '51820', 'WILLIAMSBURG CITY': '51830', 'WINCHESTER CITY': '51840'}, 'WA': {'ADAMS': '53001', 'ASOTIN': '53003', 'BENTON': '53005', 'CHELAN': '53007', 'CLALLAM': '53009', 'CLARK': '53011', 'COLUMBIA': '53013', 'COWLITZ': '53015', 'DOUGLAS': '53017', 'FERRY': '53019', 'FRANKLIN': '53021', 'GARFIELD': '53023', 'GRANT': '53025', 'GRAYS HARBOR': '53027', 'ISLAND': '53029', 'JEFFERSON': '53031', 'KING': '53033', 'KITSAP': '53035', 'KITTITAS': '53037', 'KLICKITAT': '53039', 'LEWIS': '53041', 'LINCOLN': '53043', 'MASON': '53045', 'OKANOGAN': '53047', 'PACIFIC': '53049', 'PEND OREILLE': '53051', 'PIERCE': '53053', 'SAN JUAN': '53055', 'SKAGIT': '53057', 'SKAMANIA': '53059', 'SNOHOMISH': '53061', 'SPOKANE': '53063', 'STEVENS': '53065', 'THURSTON': '53067', 'WAHKIAKUM': '53069', 'WALLA WALLA': '53071', 'WHATCOM': '53073', 'WHITMAN': '53075', 'YAKIMA': '53077'}, 'WV': {'BARBOUR': '54001', 'BERKELEY': '54003', 'BOONE': '54005', 'BRAXTON': '54007', 'BROOKE': '54009', 'CABELL': '54011', 'CALHOUN': '54013', 'CLAY': '54015', 'DODDRIDGE': '54017', 'FAYETTE': '54019', 'GILMER': '54021', 'GRANT': '54023', 'GREENBRIER': '54025', 'HAMPSHIRE': '54027', 'HANCOCK': '54029', 'HARDY': '54031', 'HARRISON': '54033', 'JACKSON': '54035', 'JEFFERSON': '54037', 'KANAWHA': '54039', 'LEWIS': '54041', 'LINCOLN': '54043', 'LOGAN': '54045', 'MCDOWELL': '54047', 'MARION': '54049', 'MARSHALL': '54051', 'MASON': '54053', 'MERCER': '54055', 'MINERAL': '54057', 'MINGO': '54059', 'MONONGALIA': '54061', 'MONROE': '54063', 'MORGAN': '54065', 'NICHOLAS': '54067', 'OHIO': '54069', 'PENDLETON': '54071', 'PLEASANTS': '54073', 'POCAHONTAS': '54075', 'PRESTON': '54077', 'PUTNAM': '54079', 'RALEIGH': '54081', 'RANDOLPH': '54083', 'RITCHIE': '54085', 'ROANE': '54087', 'SUMMERS': '54089', 'TAYLOR': '54091', 'TUCKER': '54093', 'TYLER': '54095', 'UPSHUR': '54097', 'WAYNE': '54099', 'WEBSTER': '54101', 'WETZEL': '54103', 'WIRT': '54105', 'WOOD': '54107', 'WYOMING': '54109'}, 'WI': {'ADAMS': '55001', 'ASHLAND': '55003', 'BARRON': '55005', 'BAYFIELD': '55007', 'BROWN': '55009', 'BUFFALO': '55011', 'BURNETT': '55013', 'CALUMET': '55015', 'CHIPPEWA': '55017', 'CLARK': '55019', 'COLUMBIA': '55021', 'CRAWFORD': '55023', 'DANE': '55025', 'DODGE': '55027', 'DOOR': '55029', 'DOUGLAS': '55031', 'DUNN': '55033', 'EAU CLAIRE': '55035', 'FLORENCE': '55037', 'FOND DU LAC': '55039', 'FOREST': '55041', 'GRANT': '55043', 'GREEN': '55045', 'GREEN LAKE': '55047', 'IOWA': '55049', 'IRON': '55051', 'JACKSON': '55053', 'JEFFERSON': '55055', 'JUNEAU': '55057', 'KENOSHA': '55059', 'KEWAUNEE': '55061', 'LA CROSSE': '55063', 'LAFAYETTE': '55065', 'LANGLADE': '55067', 'LINCOLN': '55069', 'MANITOWOC': '55071', 'MARATHON': '55073', 'MARINETTE': '55075', 'MARQUETTE': '55077', 'MENOMINEE': '55078', 'MILWAUKEE': '55079', 'MONROE': '55081', 'OCONTO': '55083', 'ONEIDA': '55085', 'OUTAGAMIE': '55087', 'OZAUKEE': '55089', 'PEPIN': '55091', 'PIERCE': '55093', 'POLK': '55095', 'PORTAGE': '55097', 'PRICE': '55099', 'RACINE': '55101', 'RICHLAND': '55103', 'ROCK': '55105', 'RUSK': '55107', 'ST CROIX': '55109', 'SAUK': '55111', 'SAWYER': '55113', 'SHAWANO': '55115', 'SHEBOYGAN': '55117', 'TAYLOR': '55119', 'TREMPEALEAU': '55121', 'VERNON': '55123', 'VILAS': '55125', 'WALWORTH': '55127', 'WASHBURN': '55129', 'WASHINGTON': '55131', 'WAUKESHA': '55133', 'WAUPACA': '55135', 'WAUSHARA': '55137', 'WINNEBAGO': '55139', 'WOOD': '55141'}, 'WY': {'ALBANY': '56001', 'BIG HORN': '56003', 'CAMPBELL': '56005', 'CARBON': '56007', 'CONVERSE': '56009', 'CROOK': '56011', 'FREMONT': '56013', 'GOSHEN': '56015', 'HOT SPRINGS': '56017', 'JOHNSON': '56019', 'LARAMIE': '56021', 'LINCOLN': '56023', 'NATRONA': '56025', 'NIOBRARA': '56027', 'PARK': '56029', 'PLATTE': '56031', 'SHERIDAN': '56033', 'SUBLETTE': '56035', 'SWEETWATER': '56037', 'TETON': '56039', 'UINTA': '56041', 'WASHAKIE': '56043', 'WESTON': '56045'}, 'AS': {'UNKNOWN': '60010', 'MANUA': '60020', 'OFU': '60020', 'OLOSEGA': '60020', 'TUTUILA': '60050'}, 'MP': {'COCOS ISLAND': '66010', 'ROTA': '69100', 'SAIPAN': '69110', 'AGUIJAN': '69120', 'TINIAN': '69120'}, 'GU': {'GUAM': '66010'}, 'PR': {'ADJUNTAS': '72001', 'AGUADA': '72003', 'AGUADILLA': '72005', 'AGUAS BUENAS': '72007', 'AIBONITO': '72009', 'ANASCO': '72011', 'ARECIBO': '72013', 'ARROYO': '72015', 'BARCELONETA': '72017', 'BARRANQUITAS': '72019', 'BAYAMON': '72021', 'CABO ROJO': '72023', 'CAGUAS': '72025', 'CAMUY': '72027', 'CANOVANAS': '72029', 'CAROLINA': '72031', 'CATANO': '72033', 'CAYEY': '72035', 'CEIBA': '72037', 'CIDRA': '72041', 'COAMO': '72043', 'COMERIO': '72045', 'COROZAL': '72047', 'CULEBRA': '72049', 'DORADO': '72051', 'FAJARDO': '72053', 'FLORIDA': '72054', 'GUANICA': '72055', 'GUAYAMA': '72057', 'GUAYANILLA': '72059', 'GUAYNABO': '72061', 'GURABO': '72063', 'HATILLO': '72065', 'HORMIGUEROS': '72067', 'ISABELA': '72071', 'JAYUYA': '72073', 'JUANA DIAZ': '72075', 'JUNCOS': '72077', 'LAJAS': '72079', 'LARES': '72081', 'LAS MARIAS': '72083', 'LAS PIEDRAS': '72085', 'LOIZA': '72087', 'LUQUILLO': '72089', 'MANATI': '72091', 'MARICAO': '72093', 'MAUNABO': '72095', 'MAYAGUEZ': '72097', 'MOCA': '72099', 'MOROVIS': '72101', 'NABUABO': '72103', 'NARANJITO': '72105', 'OROCOVIS': '72107', 'PATILLAS': '72109', 'PENUELAS': '72111', 'PONCE': '72113', 'QUEBRADILLAS': '72115', 'RINCON': '72117', 'RIO GRANDE': '72119', 'SABANA GRANDE': '72121', 'SALINAS': '72123', 'SAN GERMAN': '72125', 'SAN JUAN': '72127', 'SAN LORENZO': '72129', 'SAN SABASTIAN': '72131', 'SANTA ISABEL': '72133', 'TOA ALTA': '72135', 'TOA BAJA': '72137', 'TRUJILLO ALTO': '72139', 'UTUADO': '72141', 'VEGA ALTA': '72143', 'VEGA BAJA': '72145', 'VIEQUES': '72147', 'VILLALBA': '72149', 'YABUCOA': '72151', 'YAUCO': '72153'}, 'VI': {'ST. CROIX': '78010', 'ST. JOHN': '78020', 'ST. THOMAS': '78030'}}

In [0]:
def add_county_fip(df):
    fip = []
    look_up_dict=dict_states
    for ind in df.index:
        county = (df["COUNTY_NAME"][ind])
        state = (df["STATE_ALPHA"][ind])
        state_dict = look_up_dict[state]
        code = state_dict[county]
        fip.append(str(code))
    df["COUNTY_FIP"]=fip
    return df

def value_convert(df_economic):
  arr = df_economic["VALUE"]
  arr_out = []
  for val in arr:
    string_val = str(val)
    string_val=string_val.replace(",", "")
    if string_val == '(D)' or string_val == '(z)' or string_val == '(Z)':
      string_val = "0"
    arr_out.append(float(string_val))
  df_economic["VALUE"] = arr_out
  return df_economic

def make_widget_year(frame):
  YEAR = widgets.Dropdown(
      options=sorted(frame.YEAR.unique()),
      description="YEAR:",
      disabled=False)
  display(YEAR)
  return YEAR

def make_plot(df_economcic_sub):
  fig = px.choropleth_mapbox(df_economcic_sub, geojson=counties, locations='COUNTY_FIP', color='VALUE',
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 40.5, "lon": -74.871826},
                           opacity=0.8, hover_name="COUNTY_NAME"
                          )
  fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
  #fig = go.FigureWidget()
  #trace = go.Choroplethmapbox(geojson=counties, locations=df_economcic_sub.COUNTY_FIP, z=df_economcic_sub.VALUE,
  #                                  colorscale="Viridis", 
  #                                  marker_opacity=.8, marker_line_width=.5)
  #fig.update_layout(mapbox_style="carto-positron",mapbox_zoom = 6,
  #                mapbox_center = {"lat": 40.5, "lon": -74.871826})
  #
  #fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
  #fig.add_trace(trace)
  return fig  

def check(in_put, dataframe):    
  SHORT_DESC_ARR = dataframe.SHORT_DESC.unique()
  if in_put in SHORT_DESC_ARR:
      df_economcic_sub = dataframe[(dataframe["SHORT_DESC"] == (in_put))]
  else:
      df_economcic_sub = dataframe[(dataframe["YEAR"] == int(in_put))]
  arr_year = df_economcic_sub.YEAR.unique()
  arr_year_frames = {}
  if(len(arr_year) > 1):
      for year in arr_year:
          df_economcic_sub_year = df_economcic_sub[(df_economcic_sub["YEAR"] == (year))]
          arr_year_frames[year]=(df_economcic_sub_year)
      widget_year = make_widget_year(df_economcic_sub)
      return arr_year_frames, widget_year
  else:
      cat = str(df_economcic_sub.iloc[0]['SHORT_DESC']) 
      year = str(df_economcic_sub.iloc[0]['YEAR']) 
      string = cat + " " + year
      print(string)
      fig = make_plot(df_economcic_sub)
      fig.show()
      return (0,0)

In [0]:
df_temp = df_economic
COUNTY_ARR = dict_states[STATE].keys()
for ind in df_economic.index:
  COUNTY = df_economic["COUNTY_NAME"][ind]
  if COUNTY not in COUNTY_ARR:
    df_economic.at[ind, 'COUNTY_NAME'] = "drop"
df_economic = df_economic[(df_economic["COUNTY_NAME"] != 'drop')]
df_economic=add_county_fip(df_economic)
df_economic=value_convert(df_economic)

# Using the interactive portion
### As I am still working out the kinks, this is a multistep process. Please follow the below steps to see data plotted on a map

1.   Begin by running the below box with the comment **Box 1** - this will present the first interactive drop down to allow you to select the data by description
2.   Next run **Box 2** which will either present the plot if there is only a single year for the data. Otherwise, it will present an additional drop down of year 
3.  Finally, after selecting a year run **Box 3** which will present the output map



In [0]:
#Box 1
SHORT_DESC = widgets.Dropdown(
      options=sorted(df_economic.SHORT_DESC.unique()),
      description='SHORT DESCRIPTION:',
      disabled=False)
display(SHORT_DESC)

In [0]:
#Box 2
#after selecting in above menu, run me - I will either present a map or offer you a list of years
entry_A = SHORT_DESC.value
frames_A,YEAR_A = check(entry_A, df_economic)

In [0]:
#Box 3
#if map did not display above, select a year from the above dropdown and run me
entry = YEAR_A.value
frames,YEAR = check(entry, frames_A[entry])